# Setup

In [1]:
# MAGIC

# %load_ext nb_black

%matplotlib inline
# %matplotlib notebook

# ---

# IMPORT

import os
import sys
import csv
import json
import copy
import pickle

# import pprint
from pprint import pprint

import math
import random as rn

import numpy as np
import pandas as pd
import scipy.linalg as la
# from scipy.stats import rankdata

from collections import Counter
from collections import defaultdict

import itertools
from itertools import permutations
# from itertools import combinations

import sqlite3
from sqlite3 import Error

import timeit

# import pytz
from datetime import datetime
# from datetime import timedelta

import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import matplotlib, cm

import seaborn as sns
sns.set_style('white')

import networkx as nx
# from networkx.algorithms import bipartite

# import tensorflow as tf
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

# ---

# PLOT

# List of matplotlib styles
# print(plt.style.available)
# plt.style.use('classic')
# plt.style.use('dark_background')
# plt.style.use('fast')
# plt.style.use('fivethirtyeight')
# plt.style.use('ggplot')
# plt.style.use('grayscale')
# plt.style.use('seaborn')
# plt.style.use('seaborn-bright')
# plt.style.use('seaborn-colorblind')
# plt.style.use('seaborn-dark')
# plt.style.use('seaborn-dark-palette')
# plt.style.use('seaborn-darkgrid')
# plt.style.use('seaborn-deep')
# plt.style.use('seaborn-muted')
# plt.style.use('seaborn-notebook')
# plt.style.use('seaborn-paper')
# plt.style.use('seaborn-pastel')
# plt.style.use('seaborn-poster')
# plt.style.use('seaborn-talk')
# plt.style.use('seaborn-ticks')
# plt.style.use('seaborn-white')
# plt.style.use('seaborn-whitegrid')
# plt.style.use('tableau-colorblind10')
# plt.style.use('Solarize_Light2')

# Publication plot style
# plt.rc('figure', figsize=(4, 3))  # or (8, 6)
# plt.rc('font', family='serif')
# plt.rc('font', family='monospace')
# plt.rc('font', size=12)
# plt.rc('xtick', labelsize=10)
# plt.rc('ytick', labelsize=10)
# plt.rc('xtick', labelsize='x-small')
# plt.rc('ytick', labelsize='x-small')
# plt.rc('savefig', dpi=400)
# plt.rc('savefig', bbox_inches='tight')

# My plot style
# mpl.rc('font', **{'family': 'sans-serif', 'sans-serif': ['Arial']})
# mpl.rc('font', size=16)  # default text sizes
# plt.rc('figure', titlesize=16)  # fontsize of the figure title
# plt.rc('axes', titlesize=16)  # fontsize of the axes title
# mpl.rc('axes', labelsize=14)  # fontsize of the x and y labels
# plt.rc('legend', fontsize=14)  # legend fontsize
# mpl.rc('xtick', labelsize=14, color='#222222')  # fontsize of the tick labels
# mpl.rc('ytick', labelsize=14, color='#222222')  # 222222 is dark grey
# mpl.rc('xtick.major', size=6, width=1)
# mpl.rc('xtick.minor', size=3, width=1)
# mpl.rc('ytick.major', size=6, width=1)
# mpl.rc('ytick.minor', size=3, width=1)
# mpl.rc('axes', linewidth=1, edgecolor='#222222', labelcolor='#222222')
# mpl.rc('text', usetex=False, color='#222222')
# mpl.rc('text', usetex=True, color='#222222')

# Reset plot style
# plt.rcdefaults()

# Seaborn library config
# sns.set(color_codes=True)
# sns.set(rc={'figure.figsize':(5,5)})

# --- METHOD ---

# -- DB ---


def db_connect(db):
    """
    Connect to the input database
    """
    try:
        conn = sqlite3.connect(db)
        return conn
    except Error as e:
        print(e)
    return None


def db_close(conn):
    """
    Close the connection to the database
    """
    conn.close()


def db_row_count(conn, table_name, print_out=False):
    """
    Calculate the number of entries in the input table
    """
    cur = conn.cursor()
    cur.execute('SELECT COUNT(*) FROM {}'.format(table_name))
    count = cur.fetchall()
    if print_out:
        print('\n|DB|={}'.format(count[0][0]))
    return count[0][0]


def db_create(db, query):
    """
    Create a database based using the input CREATE query
    """
    conn = db_connect(db)
    try:
        cur = conn.cursor()
        cur.execute(query)
    except Error as e:
        print(e)
    conn.close()


def db_insert_many(db, query, data):
    """
    Insert many (more than one) row to a table of a database
    """
    conn = db_connect(db)
    try:
        cur = conn.cursor()
        cur.executemany(query, data)
        conn.commit()
    except Error as e:
        print(e)
    conn.close()


# --- System ---


def dir_walk(path, ext='', save=False):
    """
    Walk through a directory, find all the file with specified extension
    """
    # set ext to a specific extension if needed
    f = []
    for root, dirs, files in os.walk(path):
        for file in files:
            # relative path of file
            relative_path = os.path.join(root, file)
            # extention (type of file)
            ext_of_file = os.path.splitext(relative_path)[-1].lower()[1:]
            # if extension is set and equal to what we want
            if ext != '' and ext_of_file == ext:
                f.append(os.path.abspath(relative_path))
            # if extension is not set add the file anyway
            else:
                f.append(os.path.abspath(relative_path))
    f.sort()
    if save:
        np.savetxt('files.csv', f, delimiter=',', fmt='%s')
    return f


def line_count(filename):
    """
    Count the number of lines in the input file
    """
    # count = 0
    # for line in open(filename).xreadlines(  ): count += 1
    # return count
    # OR
    return len(open(filename).readlines())


# --- Colors ---


def colors_create(number_of_colors=1, color_map='Wistia', output=False):
    """
    create a series of colors from the selected spectrum, e.g., Wistia [cold to hot]
    """
    color_list = []
    cmap = cm.get_cmap(color_map, number_of_colors)
    for i in range(cmap.N):
        # will return rgba, we take only first 3 so we get rgb
        rgb = cmap(i)[:3]
        # print(matplotlib.colors.rgb2hex(rgb))
        color_list.append(matplotlib.colors.rgb2hex(rgb))
    if output:
        for i in range(len(color_list)):
            plt.scatter(i, 1, c=color_list[i], s=20)
    return color_list


# --- Data Structure ---


def dict_add(dictionary, key, value):
    """
    Add a key:value to dictionary if key does not already exist
    """
    if key not in dictionary:
        dictionary[key] = value


def dict_lookup(dictionary, key):
    """
    Search the given KEY in dictionary ...
    found -> return its value (could be an index assign to that value)
    not found -> add the key and return its value (which is a new index)
    useful for creating hash table of KEY->INDEX
    """
    value = 0
    if key not in dictionary:
        value = len(dictionary)
        dictionary[key] = value
    else:
        value = dictionary.get(key)
    return value


def label_amend(label_list, input_label, end=True):
    """
    Add (or remove) an input label to list of labels
    Type of input_label can be integer or string
    List of labels are assume to have a name like:
    ['folder/file.extension','folder/file.extension']
    Output looks like:
    ['folder/file_label.extension','folder/file_label.extension']
    """
    # Fist check to see if list and input_label are not None
    if len(label_list) > 0 and len(input_label) > 0:
        # Then amend all labels in the label list
        if end:
            # Can be use to amend file name (before the file type)
            # By default adds input label to end of all labels in the list
            label_list = [
                label.split('.')[0] + '_' + str(input_label) + '.' +
                label.split('.')[1] for label in label_list
            ]
        else:
            # Can be use to amend folder of a file and adds a pre-fix name to the folder
            # We assume we onle have one sub-folder-level (or character "/") in the name
            label_list_new = []
            root = os.getcwd()
            for label in label_list:
                path = os.path.join(
                    root,
                    label.split('/')[0] + '/' + str(input_label)
                )
                # Making sure the folder exist, otherwise create it
                os.makedirs(path, exist_ok=True)
                label_list_new.append(
                    label.split('/')[0] + '/' + str(input_label) + '/' +
                    label.split('/')[1]
                )
            label_list = label_list_new[:]
    return label_list


def list_intersection(lst1, lst2, version=4):
    """
    Intersection of two list or all common elements of two lists
    """
    if version == 1:
        return [value for value in lst1 if value in lst2]
    elif version == 2:
        return list(set(lst1) & set(lst2))
    elif version == 3:
        if len(lst1) > len(lst2):
            return set(lst1).intersection(lst2)
        else:
            return set(lst2).intersection(lst1)
    elif version == 4:  # O(n)
        temp = set(lst2)
        return [value for value in lst1 if value in temp]


def rank(x, return_rank=False):
    """
    Rank items of a list from largest to smallest value
    and return a list of [(index,value,rank)]
    """
    # Input is list
    if isinstance(x, list):
        # Convert to series
        s = pd.Series(x)
    # Input is series
    if isinstance(x, pd.Series):
        # Only sort based on the index
        s = x.sort_index()
    # Input is 2D array
    if isinstance(x, np.ndarray):
        s = pd.Series(x.flatten())
    # Input is dictionary
    if isinstance(x, dict):
        s = pd.Series(x, index=sorted(x.keys()))
    # Rank the data
    ranked = s.rank(method='dense', ascending=False).astype(int).sort_values()
    # If input was 2D array change index to tuple (i,j) of matrix
    if isinstance(x, np.ndarray):
        temp = np.unravel_index(ranked.index, x.shape)
        ranked.index = list(zip(temp[0],temp[1]))
    # If the rank values are needed then return entire series
    if return_rank:
        return ranked
    # Otherwise return ranked index of items
    return list(ranked.index)


# Test for rank
assert rank(
    {
        0: 2,
        1: 4,
        2: 6,
        3: 8,
        4: 10,
        5: 9,
        6: 7,
        7: 7,
        8: 7,
        9: 0,
        10: 1,
        11: 2
    }
) == [4, 5, 3, 6, 7, 8, 2, 1, 0, 11, 10, 9]


def breakdown(lst, num):
    """
    Breakdown a list into chunks of sub-list with size of n
    """
    # pprint(list(range(len(lst))))
    # pprint(lst)

    # Sort the list (high -> low)
    # Rank the sorted list
    ranks = pd.Series(lst).rank(method='dense',
                                ascending=False).astype(int).sort_values()
    # Divide the ranks into chunk of desired size
    chunks = [list(ranks.iloc[i:i + num]) for i in range(0, len(ranks), num)]
    # Dictionary of {rank : indices}
    rank_idx = {i: set() for i in set(ranks)}
    for idx, rank in ranks.items():
        # print(f'{rank} : {idx}')
        rank_idx[rank].add(idx)
    # Create a new chunk, but index of high ranks to low ranks
    bd = []
    for chunk in chunks:
        lst_temp = []
        for rank in chunk:
            # Picl a random index from the selected rank
            idx = rn.sample(rank_idx[rank], 1)[0]
            lst_temp.append(idx)
            rank_idx.get(rank).remove(idx)
        bd.append(lst_temp)
    return bd


# --- Save & load ---


def fig_save(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    """
    Method for saving figure
    """
    path = os.path.join(IMAGES_PATH, fig_id + '.' + fig_extension)
    print('saving figure ...', fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


def dict_save(data, file='data', method='n', sort=False):
    """
    Save input dictionary using different method:
        n: numpy (npy) -> good for saving the data type
        c: pandas (csv) -> good for looking at data after saving and sorting
        j: json -> also good for looking at data (simple key type) and sorting
        p: pickle -> should be fastest, good for simple data type
    """
    # npy
    if method == 'n':
        filename = file + '.npy'
        np.save(filename, data)
    # csv
    elif method == 'c':
        filename = file + '.csv'
        if not sort:
            pd.DataFrame.from_dict(data, orient='index'
                                   ).to_csv(filename, header=False)
        else:
            pd.DataFrame.from_dict(data, orient='index').sort_index(
                axis=0
            ).to_csv(filename, header=False)
    # json
    elif method == 'j':
        filename = file + '.json'
        with open(filename, 'w') as fp:
            if not sort:
                json.dump(data, fp)
            else:
                json.dump(data, fp, sort_keys=True, indent=4)
    # pickle
    elif method == 'p':
        filename = file + '.p'
        with open(filename, 'wb') as fp:
            pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)


def dict_load(file='data', method='n'):
    """
    Load input file into a dictionary using different method:
        n: numpy (npy)
        c: pandas (csv)
        j: json
        p: pickle
    """
    data = {}
    # npy
    if method == 'n':
        filename = file + '.npy'
        data = np.load(filename, allow_pickle='True').item()
    # csv
    elif method == 'c':
        filename = file + '.csv'
        data = pd.read_csv(filename, header=None,
                           index_col=0).T.to_dict('records')[0]
    # json
    elif method == 'j':
        filename = file + '.json'
        with open(filename, 'r') as fp:
            data = json.load(fp)
    # pickle
    elif method == 'p':
        filename = file + '.p'
        with open(filename, 'rb') as fp:
            data = pickle.load(fp)
    return data


def list_save(
    input_list,
    file_name='list',
    file_postname='',
    file_extension='csv',
    delimiter=',',
    replace=True,
):
    """
    Method for saving list
    """
    if len(postname) > 0:
        file_name = file_name + '_' + file_postname
    file_path = os.path.join(OUTPUT_PATH, file_name + '.' + file_extension)
    if os.path.exists(file_input):
        print('file already exist.')
        if replace:
            print('overwriting ... ')
            np.savetxt(file_path, input_list, delimiter=delimiter, fmt='%s')
        else:
            print('saving ... ')
            file_path = os.path.join(
                OUTPUT_PATH, file_name + '_new.' + file_extension
            )
            np.savetxt(file_path, input_list, delimiter=delimiter, fmt='%s')
        print('done!')
    else:
        print('saving ... ')
        np.savetxt(file_path, input_list, delimiter=delimiter, fmt='%s')
        print('done!')


# --- Linear Algebra ---

# def array_top_n(arr, top_N=1):
#     """
#     find top 'N' values in 2d numpy array
#     """
#     idx = np.argpartition(arr, arr.size - top_N, axis=None)[-top_N:][::-1]
#     result = np.column_stack(np.unravel_index(idx, arr.shape))
#     return [(e[0], e[1]) for e in result]


def top_n(arr, top_N=1, index=True):
    """
    find top 'N' values of 1d numpy array or list
    Return the index of top values (if index == True) or index and value as tuple
    """
    idx = np.argsort(arr)[::-1][:top_N]
    if index:
        return idx
    else:
        return [(e, arr[e]) for e in idx]


def array_top_n(arr, top_N=1):
    """
    find top 'N' values in 2d numpy array
    """
    idx = (-arr).argsort(axis=None, kind='mergesort')
    # idx = (-arr).argsort(axis=None, kind='mergesort')[:top_N]
    result = np.vstack(np.unravel_index(idx, arr.shape)).T
    return [(e[0], e[1]) for e in result]


def matrix_print(M, out_int=True):
    """
    Print input matrix in terminal, without any cut-off
    """
    for i, element in enumerate(M):
        if out_int:
            print(*element.astype(int))
        else:
            print(*element)


# --- FOLDERS ---

# --- Common Folders ---
# ROOT_PATH = '.'
ROOT_PATH = os.getcwd()
DATA_PATH = os.path.join(ROOT_PATH, 'data')
os.makedirs(DATA_PATH, exist_ok=True)
IMAGE_PATH = os.path.join(ROOT_PATH, 'image')
os.makedirs(IMAGE_PATH, exist_ok=True)
# --- Project Specific Folders ---
DB_PATH = os.path.join(ROOT_PATH, 'db')
os.makedirs(DB_PATH, exist_ok=True)
FILE_PATH = os.path.join(ROOT_PATH, 'file')
os.makedirs(DB_PATH, exist_ok=True)
NET_PATH = os.path.join(ROOT_PATH, 'network')
os.makedirs(NET_PATH, exist_ok=True)
HITS_PATH = os.path.join(ROOT_PATH, 'hits')
os.makedirs(HITS_PATH, exist_ok=True)

# Snippet

#### Time takes for a part of code to run

In [ ]:
# start_time = timeit.default_timer()
# CODE
# elapsed_time = timeit.default_timer() - start_time

#### Save a dictionary to **CSV** file

In [ ]:
# with open('dict.csv', 'w') as csv_file:
#     writer = csv.writer(csv_file)
#     for key, value in time_out_degrees.items():
#         writer.writerow([key, value])

#### Save and load a dictionary using **PICKLE**

In [ ]:
# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('filename.pickle', 'rb') as handle:
#     unserialized_data = pickle.load(handle)

#### Save a list to **TXT** file

In [ ]:
# np.savetxt(file_path, list_name, delimiter=',', fmt='%s')

#### Breakdown a list into chunk of size **n**

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# pprint(list(chunks(list(range(10, 75)), 10)))

In [ ]:
n = 10
lst = list(range(10, 75))
lst = [lst[i:i + n] for i in range(0, len(lst), n)]

# pprint(lst)

#### Rank 2D array

In [40]:
from scipy.stats import rankdata

arr = np.random.randint(10, size=(3, 5))
print(arr)

ranks=rankdata(arr.max()-arr,method='dense').reshape(arr.shape)
print(ranks)

[[6 6 6 8 4]
 [5 7 3 8 9]
 [8 0 8 5 1]]
[[4 4 4 2 6]
 [5 3 7 2 1]
 [2 9 2 5 8]]


# Dataset

## Create

In [ ]:
def dataset_db(file_input=['data'], file_output=['uiuc.db'], output=True):
    """
    Read log files of each user, detect user_hash based on the log names
    then rename the folder to the user_id and extract data from log files
    """

    # Paths
    DATA = os.path.join(ROOT_PATH, file_input[0])
    DB = os.path.join(DB_PATH, file_output[0])

    # Reading files
    files = dir_walk(DATA)

    # Dictionary {User : MAC}
    user_mac = {}
    for f in files:
        sp = f.split('/')
        # Second to last is user_id [1-28]
        u = int(sp[-2])
        # Last is the file_name = datetime + mac
        d, mac = sp[-1].split('.')
        dict_add(user_mac, u, mac)

    # Sort user_mac based on user_id (key)
    user_mac = sorted([(int(k), v) for k, v in user_mac.items()])

    # Save user_mac to DB
    query = f'''
    CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    mac TEXT NOT NULL
    );
    '''
    db_create(DB, query)
    query = 'INSERT INTO users VALUES (?,?)'
    db_insert_many(DB, query, user_mac)

    # Read BT and WiFI data
    # error_list = [] # TEST
    user_b = defaultdict(list)  # Bluetooth
    user_w = defaultdict(list)  # WiFi
    b_mac = []  # Bluetooth MAC list
    w_mac = []  # WiFi MAC list

    # Add BT users to the list of bluetooth devices
    # They are not needed in WiFi
    # Because they only see access points MAC in logs
    b_mac = [u[1] for u in user_mac]

    for file in files:
        is_b = False
        is_w = False
        sp = file.split('/')
        u = int(sp[-2])
        d, mac = sp[-1].split('.')
        # Check the type of log file
        if d[0] == 's':  # BT
            is_b = True
        else:  # WiFi
            is_w = True

        # Read file line by line -> save to DF -> convert DF column to series
        df = pd.read_csv(file, names=['line'])
        lines = df.line
        last_time = ''

        for i in range(len(lines)):
            # Length of line with time info = 21
            # Length of line with mac info = 40
            if len(lines[i]) < 40:  # We read a line with time info ...
                # String format
                # last_time = lines[i]
                # Datetime format
                last_time = datetime.strptime(lines[i][2:], '%m-%d-%Y %H:%M:%S')
                continue
            else:  # We read a line with mac address ...
                # If last_time is out of range i.e. < 2010-2-25 then skip until read a valid one
                if last_time < datetime(2010, 2, 25):
                    continue
                # If the last_time was valid and the line is a BT or wifi do ...
                if is_b:  # Bluetooth
                    b_index = 0
                    # b_index = -1  # TEST: changed to -1 to detect the possible errors
                    if lines[i] not in b_mac:
                        # Add new MAC to list
                        b_mac.append(lines[i])
                        # Index of new mac = (len - 1) of list, becasue it was the latest added element
                        b_index = len(b_mac) - 1
                    else:
                        b_index = b_mac.index(lines[i])
                    # Add new BT log to dictionary {user:[(time,mac)]
                    user_b[u].append((last_time, b_index))
                    # TEST
                    # if b_index < 0:
                    # if last_time < datetime(2010,2,24):
                    # error_list.append((file,lines[i],last_time))

                else:  # WiFi
                    w_index = 0
                    # w_index = -1  # TEST
                    if lines[i] not in w_mac:
                        # Add new MAC to list
                        w_mac.append(lines[i])
                        # Index of new mac = (len - 1) of list becasue it is the last added element
                        w_index = len(w_mac) - 1
                    else:
                        w_index = w_mac.index(lines[i])
                    user_w[u].append((last_time, w_index))
                    # TEST
                    # if w_index < 0:
                    # if last_time < datetime(2010,2,24):
                    # error_list.append((file,lines[i],last_time))
    # TEST : save the errors
    # np.savetxt('error_list.csv', error_list, delimiter=',', fmt='%s')

    if output:
        print('# BT devices:', len(b_mac))
        print('# WiFi devices:', len(w_mac))

    # Save {BT_id : BT_MAC} to DB
    query = f'''
    CREATE TABLE IF NOT EXISTS bluetooth (
    id INTEGER PRIMARY KEY,
    mac TEXT NOT NULL
    );
    '''
    db_create(DB, query)
    b_mac_insert = [(i, b_mac[i]) for i in range(len(b_mac))]
    query = 'INSERT INTO bluetooth VALUES (?,?)'
    db_insert_many(DB, query, b_mac_insert)

    # Save {WiFi_id : WiFi_MAC} to DB
    query = f'''
    CREATE TABLE IF NOT EXISTS wifi (
    id INTEGER PRIMARY KEY,
    mac TEXT NOT NULL
    );
    '''
    db_create(DB, query)
    w_mac_insert = [(i, w_mac[i]) for i in range(len(w_mac))]
    query = 'INSERT INTO wifi VALUES (?,?)'
    db_insert_many(DB, query, w_mac_insert)

    # Save BT and WiFi logs to DB with following columns
    # user_node, bluetooth_node / ap_node , time, bluetooth = 0 / wifi = 1
    query = f'''
    CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY,
    user INTEGER NOT NULL,
    mac INTEGER NOT NULL,
    time TIMESTAMP,
    wifi INTEGER
    );
    '''
    db_create(DB, query)
    interaction_insert = []
    for k, v in user_b.items():  # k = user
        # v is a list of interactions = (time, MAC)
        for item in v:
            interaction_insert.append((k, item[1], item[0], 0))
    for k, v in user_w.items():
        for item in v:
            interaction_insert.append((k, item[1], item[0], 1))
    query = 'INSERT INTO logs(user,mac,time,wifi) VALUES (?,?,?,?)'
    db_insert_many(DB, query, interaction_insert)

    if output:
        print('# interactions:', len(interaction_insert))

In [ ]:
def dataset_csv(file_input=['db/uiuc.db'],
                file_output=['db/bt.csv', 'db/wifi.csv'],
                input_label='',
                output_label='',
                time_bin=False,
                output=False,
                save=False):
    """
    Read logs from DB and divide them based on type (BT/WiFi)
    into 2 seperate dataset files bt.csv and wifi.csv
    """

    # Modify input & output file names
    if len(input_label) > 0:
        file_input = [
            label.split('.')[0] + '_' + str(input_label) + '.' +
            label.split('.')[1] for label in file_input
        ]
    if len(output_label) > 0:
        file_output = [
            label.split('.')[0] + '_' + str(output_label) + '.' +
            label.split('.')[1] for label in file_output
        ]

    # Read DB
    # BT logs : wifi=0
    # WiFi logs : wifi=1
    print('Reading DB ...')
    conn = db_connect(file_input[0])

    # wifi = 0
    # query = f'''
    # SELECT user, mac, time
    # FROM logs
    # WHERE wifi={wifi} AND time >= '2010-03-01' AND time < '2010-03-21'
    # ORDER BY time
    # '''

    query = f'''
    SELECT user, mac, time, wifi
    FROM logs
    WHERE time >= '2010-03-01' AND time < '2010-03-21'
    ORDER BY time
    '''

    df = pd.read_sql_query(query, conn)
    db_close(conn)
    print(len(df), 'records')

    # Change the time columns type : string -> datetime
    df['time'] = df['time'].astype('datetime64[ns]')
    # Create timestamp list
    times = pd.Series(sorted(df.time.unique()))
    # Range of timestamp
    if output:
        print('First timestamp: {}\nLast timestamp: {}'.format(
            times.iloc[0], times.iloc[-1]))

    # Reset the minute and the second of timestamps to zero
    # Bining the timestamps to the grangularity of hour
    if time_bin:
        df['time'] = df['time'].apply(lambda x: x.replace(minute=0, second=0))
        # Create timestamp list (again after filtering)
        times = pd.Series(sorted(df.time.unique()))

    # Distribution of timestamps throughout dataset
    # ts_count = dict(Counter(times))
    # ts_count = sorted(ts_count.items())
    # ts_count = sorted(ts_count.items(), key=lambda x: x[0], reverse=True)
    # x, y = zip(*ts_count)
    # plt.plot(x, y)

    if save:
        df[df['wifi'] == 0].iloc[:, :-1].to_csv(file_output[0], header=False, index=False)
        df[df['wifi'] == 1].iloc[:, :-1].to_csv(file_output[1], header=False, index=False)

## Read

In [ ]:
def dataset_read(
    file_input=['db/bt.csv', 'db/wifi.csv'],
    file_output=[],
    input_label='',
    output_label='',
    wifi=False,  # default = BT, if True => WiFi
    output=False,
    save=False):
    """
    Read the WiFi or BT (csv) dataset file based on wifi flag
    return Dataframe with 3 columns of (u, v, t)
    """

    # Modify input & output file names
    if len(input_label) > 0:
        file_input = [
            label.split('.')[0] + '_' + str(input_label) + '.' +
            label.split('.')[1] for label in file_input
        ]
    if len(output_label) > 0:
        file_output = [
            label.split('.')[0] + '_' + str(output_label) + '.' +
            label.split('.')[1] for label in file_output
        ]

    df = []
    if not wifi:  # BT
        df = pd.read_csv(file_input[0],
                         header=None,
                         names=['u', 'v', 't'],
                         parse_dates=['t'])
    else:  # WiFI
        df = pd.read_csv(file_input[1],
                         header=None,
                         names=['u', 'v', 't'],
                         parse_dates=['t'])

    return df

## Test

#### Create DB
Create SQLite database from raw data

In [ ]:
# dataset_db()

#### Create CSV
Times are binned with 1-hour windows

In [ ]:
# dataset_csv(time_bin=True, output=True, save=True)

#### Create CSV - Raw
Times are not binned. File names has postfix of __raw__

In [ ]:
# dataset_csv(output_label='raw', time_bin=False, output=True, save=True)

#### Read BT/WiFi data CSV file

Read the CSV file for Bluetooth records

In [ ]:
# df_bt = dataset_read()

Read the CSV file for WiFi records

In [ ]:
# df_wifi = dataset_read(wifi=True)

## Result

Read the paths of DB files for each user (in a separate folder) and save in a list   
It turned out that **29,656** records has the date less than 2010-2-25 and therefore are considered as noise, and no error was risen from the name of the devices or users   
After removing the WiFi and Bluetooth devices out of acceptable datetime range i.e. less than 2010-2-24, from **8,055** Bluetooth and **6,722** WiFi devices we are down to **7,995** Bluetooth and **6,705** WiFi devices now

# Sample Graph

## Data

Read node list and edge list data

In [ ]:
# Grab node list data hosted on Gist
# nodelist = pd.read_csv('https://gist.githubusercontent.com/brooksandrew/f989e10af17fb4c85b11409fea47895b/raw/a3a8da0fa5b094f1ca9d82e1642b384889ae16e8/nodelist_sleeping_giant.csv')
nodelist = pd.read_csv('file/nodelist_sleeping_giant.csv')

# Grab edge list data hosted on Gist
# edgelist = pd.read_csv('https://gist.githubusercontent.com/brooksandrew/e570c38bcc72a8d102422f2af836513b/raw/89c76b2563dbc0e88384719a35cba0dfc04cd522/edgelist_sleeping_giant.csv')
edgelist = pd.read_csv('file/edgelist_sleeping_giant.csv')

In [ ]:
# Preview nodelist
nodelist.head(5)

In [ ]:
# Preview edgelist
edgelist.head(10)

## Create Graph

In [ ]:
# Create empty graph
g = nx.Graph()

# Add edges and edge attributes
for i, elrow in edgelist.iterrows():
    # (node1, node2, edge attribute dict)
    # g.add_edge(elrow[0], elrow[1], attr_dict=elrow[2:].to_dict())  # deprecated
    g.add_edge(elrow[0], elrow[1])
    g[elrow[0]][elrow[1]].update(elrow[2:].to_dict())

# Add node attributes
for i, nlrow in nodelist.iterrows():
    # g.node[nlrow['id']] = nlrow[1:].to_dict()  # deprecated
    g.nodes[nlrow['id']].update(nlrow[1:].to_dict())

In [ ]:
# Preview first x edges
list(g.edges(data=True))[0:1]

In [ ]:
# Preview first x nodes
list(g.nodes(data=True))[0:5]

In [ ]:
# Summary stats
print('# of edges: {}'.format(g.number_of_edges()))
print('# of nodes: {}'.format(g.number_of_nodes()))

## Visualize Graph

In [ ]:
# Define node positions data structure (dict) for plotting
# X and Y info are from dataset
node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in g.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
dict(list(node_positions.items())[0:5])

In [ ]:
# Define node labels (dict) for plotting
# node_labels = {node: node for node in g.nodes()}

c = 0
node_labels = {}
for node in g.nodes():
    node_labels[node] = c
    # Save index in node for reference in future
    g.nodes[node]['idx'] = c
    c+= 1

# Preview of node_positions
dict(list(node_labels.items())[0:5])

In [ ]:
# Define data structure (list) of edge colors for plotting
edge_colors = [e[2]['color'] for e in g.edges(data=True)]

# Preview first 10
edge_colors[0:10]

In [ ]:
# Plot
plt.figure(figsize=(16, 12))
nx.draw(g, pos=node_positions, labels=node_labels, font_color='white', edge_color=edge_colors)
# nx.draw(g, pos=node_positions, edge_color=edge_colors, node_size=10, node_color='black')
plt.title('Graph Representation of Sleeping Giant Trail Map', size=15)
plt.show()

## Save Graph

In [ ]:
# Undirected (Original)
# nx.write_gpickle(g, 'file/sleeping_giant.gpickle')

# Directed graph
nx.write_gpickle(g.to_directed(), 'file/sleeping_giant_directed.gpickle')

## Load Graph

In [ ]:
g = nx.read_gpickle('file/sleeping_giant_directed.gpickle')

# Effective Distance

## Build

In [2]:
def effective_distance(
        file_input=['db/wifi_raw.csv', 'db/bt_raw.csv', 'db/uiuc.db'],
        file_output=['network/bt_temporal_network.gpickle'],
        label_input='',
        label_output='',
        time_max=3600,
        print_distance=False,
        print_times=False,
        save_distance=True,
        save_times=True):
    """
    Read WiFi interactions and create effective distance for
    WiFi Access Points (AP)
    """
    # Edit file names
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read WiFi data
    data = pd.read_csv(file_input[0],
                       header=None,
                       names=['user', 'mac', 'time'],
                       dtype={
                           'user': int,
                           'mac': int
                       },
                       parse_dates=['time'])

    # Number of users
    U = len(data.user.unique())

    # Add filtered time as a new column to DF
    # data['timestamp'] = data['time'].apply(lambda x: x.replace(minute=0, second=0))

    # Analyze times
    # (1)
    times = sorted(data.time.unique())  # List
    # (2)
    # times = pd.Series(sorted(data.time.unique()))  # Series
    # print('# Timestamps:', len(times))

    # Plot times-frequency
    # times_freq = data.groupby('time').size().to_frame(name='freq').reset_index()
    # fig, ax = plt.subplots(figsize=(24, 12))
    # ax.scatter(times_freq.time, times_freq.freq)
    # sns.lineplot(x='time', y='freq', data=times_freq, ax=ax)
    # sns.histplot(x='time', kde=True, data=data, ax=ax)
    # ax.set_title('Timestamp Frequency')
    # plt.show()

    # Create location ID from observing groupd of AP at the same time
    loc_id = 0
    ap_loc = {}  # {Access-Point ID : Location ID}
    loc_ap = defaultdict(set)  # {Location : {AP1, AP2, ...}}

    for user in sorted(data.user.unique()):
        for key, group in data[data.user == user].sort_values('time').groupby(
                'time'):
            # Check if any of AP (or MAC) are already registered in dictionary
            aps = set(group.mac.unique())
            ap_found = list(aps.intersection(ap_loc))
            # Nothing found -> all AP are new
            if len(ap_found) == 0:
                for ap in aps:
                    ap_loc[ap] = loc_id
                loc_ap[loc_id].update(aps)
                loc_id += 1
                continue
            # BUT if we have some are already keyed in dictionary
            loc_found = [ap_loc[ap] for ap in ap_found]
            # Do all the existing AP have same location ID ?
            if len(set(
                    loc_found)) == 0:  # if all(i == first for i in loc_found)
                # YES
                # Add the rest of AP (new ones) to that location ID as well
                loc_ap[loc_found[0]].update(aps)
                for ap in aps:
                    ap_loc[ap] = loc_found[0]
            else:
                # NO
                # Some AP have differnet locations (while they should have same)
                # Take smallest location ID (sooner defined location ID) and set to all
                loc_sel = min(loc_found)
                loc_update = set(loc_found)
                loc_update.remove(loc_sel)
                # Fix dictionaries
                for loc in loc_update:
                    for ap in loc_ap[loc]:
                        ap_loc[ap] = loc_sel
                    loc_ap[loc_sel].update(loc_ap[loc])
                    del loc_ap[loc]
                # Add/update new AP
                ap_rest = list(set(aps) - set(ap_found))
                for ap in ap_rest:
                    ap_loc[ap] = loc_sel
                loc_ap[loc_sel].update(ap_rest)

    # Clean-up the Location:AP dictionary
    loc_id = 0
    loc_ap_new = {}
    for k, v in loc_ap.items():
        if len(v) == 0:  # Empty set => location with no AP
            print('location {} has no AP : {}'.format(k, v))
        else:
            loc_ap_new[loc_id] = v
            for ap in v:
                ap_loc[ap] = loc_id
            loc_id += 1
    loc_ap = loc_ap_new

    # Save {AP : Location} and {Location : AP} dictionaries
    with open('distance/ap_location.pickle', 'wb') as handle:
        pickle.dump(ap_loc, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('distance/location_ap.pickle', 'wb') as handle:
        pickle.dump(loc_ap, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # We can think of APs - or Locations they represent - as Airports
    # Calculate the number of user move from one AP to another (or from one Location to another)
    # Think of them as flow of population between Airports
    # Create the movement porobability matrix from the flows
    # Also, create average distance between every pair of APs or Locations
    # Finally, save these information in AP-network or Location-network

    # Dataframe of [ source_ap | destination_ap | user | start_time | finish_time | gap ]
    # This DF contains all the movement AP->AP (even when location is not changed) and Location->Location
    c_sap = []
    c_dap = []
    c_sl = []  # Source Location
    c_dl = []  # Destination Location
    c_u = []
    c_st = []
    c_ft = []
    c_g = []

    # Note: all 3 dictionary of "loc_loc_user", "loc_loc_time", and "loc_loc_dist" can be calculated from DF later as well
    # This is exactly what we do for AP-AP dictionaries, where we create them from DF (only AP-AP-Distance is needed)

    # {(location, location):[user_1, user_2, ...]}
    # Useful to calculate flow OR number of travelers between locations
    loc_loc_user = defaultdict(list)
    # ---
    # {(location, location):[(time_1, time_2), ...]} which is Dictionary of List of Tuples
    # Useful to calculate timestamp of source_destination travels
    loc_loc_time = defaultdict(list)
    # ---
    # {(location, location):[travel_time_1, travel_time_2, ...]}
    # Useful to calculate average time-distance between locations
    loc_loc_dist = defaultdict(list)

    # Time -> User -> (Location,Location)
    tu_loc = defaultdict(lambda: defaultdict(set))
    # tu_loc = defaultdict(lambda: defaultdict(list))

    # Time -> User -> (AP,AP)
    tu_ap = defaultdict(lambda: defaultdict(set))
    # tu_ap = defaultdict(lambda: defaultdict(list))

    for user in sorted(data.user.unique()):
        data_temp = data[data.user ==
                         user].loc[:, ['mac', 'time']].sort_values('time')
        # First timestamp of a user
        time_1 = data_temp.iloc[0, 1]
        # First observed MAC (of AP) of a user
        macs_1 = list(data_temp[data_temp.time == time_1]['mac'].unique())
        mac_1 = macs_1[0]
        for key, group in data_temp.groupby('time'):
            # if (key - time_1).seconds != 0:  # Assuring time has moved forward
            macs_2 = list(group.mac.unique())
            mac_2 = macs_2[0]
            # IF user has moved to a new location ...
            if ap_loc[mac_1] != ap_loc[mac_2]:
                loc_loc_user[(ap_loc[mac_1], ap_loc[mac_2])].append(user)
                loc_loc_time[(ap_loc[mac_1], ap_loc[mac_2])].append(
                    (time_1, key))
                loc_loc_dist[(ap_loc[mac_1], ap_loc[mac_2])].append(
                    (key - time_1).seconds)
                # Add movement to Time-User-AP dictionary
                # We can add the condition of if time-gap is less than one hour
                # if (key - time_1).seconds <= time_max:
                tu_loc[time_1.replace(minute=0, second=0)][user].add(
                    (ap_loc[mac_1], ap_loc[mac_2]))
            # Despite the movement from on location to another
            # Record AP->AP movement in DF
            # (1)
            # macs_3 = [macs_1, macs_2]
            # combs = [p for p in itertools.product(*macs_3)]
            # (2)
            for m1 in macs_1:
                for m2 in macs_2:
                    c_sap.append(m1)
                    c_dap.append(m2)
                    c_sl.append(ap_loc[m1])
                    c_dl.append(ap_loc[m2])
                    c_u.append(user)
                    c_st.append(time_1)
                    c_ft.append(key)
                    c_g.append((key - time_1).seconds)
                    # Add movement to Time-User-Location dictionary
                    tu_ap[time_1.replace(minute=0, second=0)][user].add(
                        (m1, m2))
            time_1 = key
            macs_1 = macs_2[:]
            mac_1 = macs_1[0]

    # Create dataframe of all movements (and stay or hold)
    df = pd.DataFrame(list(zip(c_sap, c_dap, c_sl, c_dl, c_u, c_st, c_ft,
                               c_g)),
                      columns=['sap', 'dap', 'sl', 'dl', 'u', 'st', 'ft', 'g'])

    # Save movements dataframe
    df.to_csv('distance/movement.csv', header=False, index=False)

    # Read movements dataframe from file and convert datetime columns
    # df = pd.read_csv('distance/movement.csv', header=None, names=['sap', 'dap', 'sl', 'dl', 'u', 'st', 'ft', 'g'], parse_dates=['st', 'ft'])
    
    tut = {}
    for k1 in tu_ap:  # k1 = times
        user_dict = {}
        for k2 in tu_ap[k1]:  # k2 = users
            move_set = set()
            for k3 in tu_ap[k1][k2]:  # k3 = (ap,ap)
                move_set.add(k3)
            user_dict[k2] = move_set
        tut[k1] = user_dict
    tu_ap = tut
    
    # Save Time-User-AP-Movement dictionary
    with open('distance/time_user_ap.pickle', 'wb') as handle:
        pickle.dump(tu_ap, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    tut = {}
    for k1 in tu_loc:  # k1 = times
        user_dict = {}
        for k2 in tu_loc[k1]:  # k2 = users
            move_set = set()
            for k3 in tu_loc[k1][k2]:  # k3 = (location,location)
                move_set.add(k3)
            user_dict[k2] = move_set
        tut[k1] = user_dict
    tu_loc = tut
    
    # Save Time-User-Location-Movement dictionary
    with open('distance/time_user_location.pickle', 'wb') as handle:
        pickle.dump(tu_loc, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Save location dictionaries
    with open('distance/location_distance.pickle', 'wb') as handle:
        pickle.dump(loc_loc_dist, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('distance/location_user.pickle', 'wb') as handle:
        pickle.dump(loc_loc_user, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('distance/location_time.pickle', 'wb') as handle:
        pickle.dump(loc_loc_time, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Save only location-distance dictionary as csv file
    # with open('distance/location_distance.csv', 'w') as csv_file:
    # writer = csv.writer(csv_file)
    # for key, value in loc_loc_dist.items():
    # writer.writerow([*key, value])

    # DF for time where users staty in one place (i.e. AP) or moved
    # df_stay = df[df.sap == df.dap]
    # df_move = df[df.sap != df.dap]

    # ----- FLOW -----

    # Create dictinary of flow from AP to another AP {(AP1,AP2):Flow}
    # (1) freq : total flow or repeated users travels
    # (2) user : unique number of users' flow between APs
    ap_flow_freq = {}
    ap_flow_user = {}
    apff_in = {}
    apfu_in = {}
    apff_out = {}
    apfu_out = {}

    for key, group in df[df.sap == df.dap].groupby(['sap', 'dap']):
        ap_flow_freq[key] = len(group)
        ap_flow_user[key] = len(group.u.unique())

    # Unique number of users that have moved between APs
    # U = len(df[df.sap != df.dap].u.unique())

    for key, group in df[df.sap != df.dap].groupby(['sap', 'dap']):
        val_1 = len(group)
        val_2 = len(group.u.unique())
        ap_flow_freq[key] = val_1
        ap_flow_user[key] = val_2
        # Sum of incoming flow to each AP or location
        # Equals to column sum of flow matrix [A -> B]
        apff_in[key[1]] = val_1 + apff_in.get(key[1], 0)
        apfu_in[key[1]] = val_2 + apfu_in.get(key[1], 0)
        # Sum of outgoing flow from each AP or location
        apff_out[key[0]] = val_1 + apff_out.get(key[0], 0)
        apfu_out[key[0]] = val_2 + apfu_out.get(key[0], 0)
        # Create Ap -> AP temporal distance

    # Normalize flow values
    for key in ap_flow_freq:
        if key[0] == key[1]:
            # Ratio of all the stay logs over all travel flows (incoming + outgoing)
            val_3 = apff_in.get(key[0], 0) + apff_out.get(key[0], 0)
            val_4 = apfu_in.get(key[0], 0) + apfu_out.get(key[0], 0)
            ap_flow_freq[(key[0], key[1])] /= val_3
            # ap_flow_user[(key[0], key[1])] /= val_4
            ap_flow_user[(key[0], key[1])] /= U
        else:
            val_3 = apff_in.get(key[1], 0)
            val_4 = apfu_in.get(key[1], 0)
            ap_flow_freq[(key[0], key[1])] /= val_3
            # ap_flow_user[(key[0], key[1])] /= val_4
            ap_flow_user[(key[0], key[1])] /= U

    with open('distance/ap_flow_freq.pickle', 'wb') as handle:
        pickle.dump(ap_flow_freq, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('distance/ap_flow_user.pickle', 'wb') as handle:
        pickle.dump(ap_flow_user, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Location-Location-Flow dictionary
    loc_flow_freq = {}
    loc_flow_user = {}
    locff_in = {}
    locfu_in = {}
    locff_out = {}
    locfu_out = {}

    for key, group in df[df.sl == df.dl].groupby(['sl', 'dl']):
        loc_flow_freq[key] = len(group)
        loc_flow_user[key] = len(group.u.unique())

    # Unique number of users that have moved between locations
    # U = len(df[df.sl != df.dl].u.unique())

    for key, group in df[df.sl != df.dl].groupby(['sl', 'dl']):
        val_1 = len(group)
        val_2 = len(group.u.unique())
        loc_flow_freq[key] = val_1
        loc_flow_user[key] = val_2
        locff_in[key[1]] = val_1 + locff_in.get(key[1], 0)
        locfu_in[key[1]] = val_2 + locfu_in.get(key[1], 0)
        locff_out[key[0]] = val_1 + locff_out.get(key[0], 0)
        locfu_out[key[0]] = val_2 + locfu_out.get(key[0], 0)

    for key in loc_flow_freq:
        if key[0] == key[1]:
            val_3 = locff_in.get(key[0], 0) + locff_out.get(key[0], 0)
            val_4 = locfu_in.get(key[0], 0) + locfu_out.get(key[0], 0)
            loc_flow_freq[(key[0], key[1])] /= val_3
            # loc_flow_user[(key[0], key[1])] /= val_4
            loc_flow_user[(key[0], key[1])] /= U
        else:
            val_3 = locff_in.get(key[1], 0)
            val_4 = locfu_in.get(key[1], 0)
            loc_flow_freq[(key[0], key[1])] /= val_3
            # loc_flow_user[(key[0], key[1])] /= val_4
            loc_flow_user[(key[0], key[1])] /= U

    with open('distance/location_flow_freq.pickle', 'wb') as handle:
        pickle.dump(loc_flow_freq, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('distance/location_flow_user.pickle', 'wb') as handle:
        pickle.dump(loc_flow_user, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # TODO : We can visualize the values of AP and Location flows
    # Show how much is difference between normalize value of frequency and unique user's flow

    # ----- GRAPH -----

    # AP Graph (ap -> ap, weighted by distance and flow)
    graph_ap = nx.DiGraph()

    # AP-AP-Distance
    ap_ap_dist = {}

    for key, group in df[df.sap != df.dap].groupby(['sap', 'dap']):
        # gaps = [e if e <= time_max else time_max for e in group.g]
        gaps = [e for e in group.g if e <= time_max]
        # If all of time distance are more than time_max (e.g. 1 hour)
        if len(gaps) == 0:
            # (1)
            # Just add 1 time_max entry
            # gaps.append(time_max)
            # (2)
            # Add 1 time_max IF ...
            # The user was at a AP then moved home (turned Wi-Fi off - sometimes before 12 am) and turned it on in the morning (7 am)
            if len(group) == 1 and (list(group.ft)[0].dayofyear -
                                    list(group.st)[0].dayofyear) == 1 and list(
                                        group.ft)[0].hour == 7:
                # if len(group) == 1 and (list(group.ft)[0].dayofyear - list(group.st)[0].dayofyear) == 1:
                # if len(group) == 1 and list(group.st)[0].hour == 22 and list(group.ft)[0].hour == 7:
                gaps.append(time_max)
            # OR - preserve the original time distance > time_max (e.g. 1 hour)
            # gaps = [e for e in group.g]
            else:
                # If several time distance > time_max => just ignore this movement and consider it as noise in data
                continue
                # OR
                if len(group.u.unique()) == 1:  # Noise
                    continue
                else:
                    # Multiple user recorded more that 1-hour travel time -> register 1-hour as the maximum possible time travel
                    gaps.append(time_max)
        # Calculate mean of recorded time distances and add to dictionary
        ap_ap_dist[key] = int(np.mean(gaps))
        # Add the AP graph edge
        graph_ap.add_edge(key[0],
                          key[1],
                          distance=ap_ap_dist.get(key, 0),
                          flow=ap_flow_freq.get(key, 0),
                          uflow=ap_flow_user.get(key, 0))

    # Save graph
    nx.write_gpickle(graph_ap, 'distance/ap_network.gpickle')

    # Visualize graph
    # plt.figure(figsize=(16, 12))
    # nx.draw(graph_ap)
    # plt.show()

    with open('distance/ap_distance.pickle', 'wb') as handle:
        pickle.dump(ap_ap_dist, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Location Graph (location -> location, weighted by distance)
    # If we set "time_max = 3600" or 1 hour then the maximum time distance we allow by default is 1 hour
    graph_loc = nx.DiGraph()
    l1, l2 = map(list, zip(*loc_loc_dist.keys()))
    nodes = set(l1).union(l2)
    graph_loc.add_nodes_from(nodes)
    for k, v in loc_loc_dist.items():
        if len(v) > 1:  # If more than one time distance
            time_temp = [i if i <= time_max else time_max for i in v]
            loc_loc_dist[k] = int(np.mean(time_temp))
        else:  # Contains only 1 time distance
            if v[0] >= time_max:
                loc_loc_dist[k] = time_max
            else:
                loc_loc_dist[k] = v[0]
        graph_loc.add_edge(k[0],
                           k[1],
                           distance=loc_loc_dist.get(k, 0),
                           flow=loc_flow_freq.get(k, 0),
                           uflow=loc_flow_user.get(k, 0))

    # Save graph
    nx.write_gpickle(graph_loc, 'distance/location_network.gpickle')

    # Visualize graph
    # plt.figure(figsize=(16, 12))
    # nx.draw(graph_loc)
    # plt.show()

    # return df

## Read

In [3]:
def time_user_ap_movement(file_input=[
    'distance/time_user_ap.pickle', 'distance/ap_distance.pickle',
    'distance/ap_flow_freq.pickle', 'distance/ap_flow_user.pickle'
],
                          file_output=[''],
                          label_input='',
                          label_output='',
                          print_tua=False):
    """
    Read movements of users from AP to AP on each time step as a dictionary of
    { Time : User : (AP , AP) }
        Note that time is only hour of movement (minute and seconds are zero) and it may not be
        in the set of timestamps in temporal network (users have moved but did not make any contact)
    """

    tua = {}
    with open(file_input[0], 'rb') as handle:
        tua = pickle.load(handle)

    # Print info about movement dictionary
    if print_tua:
        print('# times =', len(tua.keys()), '\n---')
        for k in tua:
            print(k, end='\t')
            print('# users =', len(tua[k].items()))

    ad = {}
    apf = {}
    apu = {}
    # with open(file_input[1], 'rb') as handle:
    # apf = pickle.load(handle)
    # with open(file_input[2], 'rb') as handle:
    # apf = pickle.load(handle)
    # with open(file_input[3], 'rb') as handle:
    # apf = pickle.load(handle)

    return tua

## Run

In [ ]:
# effective_distance(print_distance=False,
#                    print_times=False,
#                    save_distance=True,
#                    save_times=True)

In [ ]:
# tau = time_user_ap_movement()
# tau = time_user_ap_movement(print_tua=True)

## Result

The result is **5912** access points and **145** locations ...    
While users moved between **238** pairs of locations (x -> y) resulting in Location-location network with 248 edges and 145 nodes.    
Dataframe of AP -> AP movement has following columns: source AP, destination AP, source Location, destination Location, starting Time, finishing Time, Gap, User

# Temporal Network

## Build

In [4]:
def temporal_bt_create(
    file_input=['db/bt.csv', 'db/uiuc.db'],
    file_output=[
        'network/bt_temporal_network.gpickle',
        'network/bt_bipartite_network.gpickle',
        'network/bt_temporal_edgelist.csv',
        'network/bt_bipartite_edgelist.csv', 'network/bt_temporal_times.csv',
        'network/bt_bipartite_times.csv', 'network/bt_temporal_nodes.csv',
        'network/bt_bipartite_nodes.csv', 'network/bt_temporal_weights.csv',
        'network/bt_bipartite_weights.csv',
        'network/bt_temporal_weights_scaled.csv',
        'network/bt_bipartite_weights_scaled.csv'
    ],
    input_label='',
    output_label='',
    output_times=False,
    output_network=True,
    save_times=True,
    save_nodes=True,
    save_weights=True,
    save_network_db=True,
    save_network_csv=True,
    save_network_file=True):
    """
    Read CSV dataset with 3 columns of: node-1, node-2, timestamp
    Then create an (aggregated) temporal network using Multiedge Directed Graph
    """

    # Modify input & output file names
    file_input = label_amend(file_input, input_label)
    file_output = label_amend(file_output, output_label)

    # Create empty network
    graph = nx.MultiDiGraph()  # G: user-user interaction network
    bipartite = nx.MultiDiGraph()  # B: (28)user-others biparite network

    # Read the dataset
    print('Reading dataset ... ')
    data = pd.read_csv(file_input[0],
                       header=None,
                       names=['user', 'mac', 'time'],
                       parse_dates=['time'])
    print('Done!\n')

    # Print Time : Frequency
    if output_times:
        times = pd.Series(sorted(data.time.unique()))
        print(len(times), 'timestamps')
        print('Timestamp : Frequency')
        for t_size in data.groupby('time').size().iteritems():
            print('{}) {} : {}'.format(times[times == t_size[0]].index[0],
                                       t_size[0], t_size[1]))
        print()

    # Create timestamp list
    times = []
    times_bipartite = []

    # Dictionary {time:{(user-1,user-2):weight}}
    time_edges = defaultdict()  # User -> User
    time_bipartite_edges = defaultdict()  # Users -> other devices

    # Group interactions by time
    for key_time, group_user_mac in data[['user',
                                          'mac']].groupby(data['time']):
        # print()
        # print('Time:\n', key_time)
        # print('Group:\n', group_user_mac)

        # Normal (co-location) graph edges in filtered timestamp
        temp_edges = []
        for key_mac, group_connection in group_user_mac.groupby(['mac'
                                                                 ])['user']:
            # print('Mac:', key_mac)
            # print('Group:', group_connection)

            # Users of connecting to filtered MAC
            temp_users = list(group_connection.unique())
            # print('Unique users:', temp_users)

            # If the ID of shared connected mac is [0-27]
            # Users directly connect to another bluetooth user
            if key_mac < 28:
                # Case where only one direct connection: user -> user
                if len(temp_users) <= 1:
                    temp_edges.append((temp_users[0], key_mac))
                    # Comment next line, if wanna have directed edges
                    temp_edges.append((key_mac, temp_users[0]))
                else:
                    # Case where more than 1 user undirectly connect together via 1 direct user -> user edge
                    # Direct edges
                    for element in temp_users:
                        temp_edges.append((element, key_mac))
                        # Uncomment next line, if wanna have undirected edges when one user observe another user directly
                        temp_edges.append((key_mac, element))
                    # Undirect edges
                    connected_users = list(permutations(temp_users, 2))
                    connected_users = [tuple(e) for e in connected_users]
                    temp_edges.extend(connected_users)
            # If users are connected to device with ID > 28
            # Meaning indirect edges with each other
            else:
                # Only consider cases of more than 1 unique user for co-location indirected edges
                if len(temp_users) > 1:
                    # Undirect edges
                    connected_users = list(permutations(temp_users, 2))
                    connected_users = [tuple(e) for e in connected_users]
                    temp_edges.extend(connected_users)

        # Add edges of current timestamp (with their strength) to dictionary
        if len(temp_edges) > 0:
            time_edges[key_time] = dict(Counter(temp_edges))

        # Bipartite graph edges
        # We don't care about MAC < 28, just want to count
        # How many times in each timestamp a user connect to a MAC
        # Dictionary {time:{(user,mac):weight}}
        bipartite_edges = {}
        # Filter connections based on (user -> mac) interaction and its weight
        # for key_mac, group_connection in group_user_mac.groupby(['mac'])['user']:
        for key_mac, group_connection in group_user_mac.groupby(
            ['mac', 'user']):
            # User connected to filtered MAC with X number of times
            # print('{}\t{}'.format(key_mac, len(group_connection)))
            bipartite_edges[key_mac] = len(group_connection)

        # Add edges of this time (with their strength) to dictionary
        time_bipartite_edges[key_time] = bipartite_edges

    # Normal (co-location) network
    l1, l2, l3, l4 = [], [], [], []  # time, node, node, weight
    for k1, v1 in time_edges.items():
        for k2, v2 in v1.items():  # k2 = edge = (u,v)
            if k2[0] != k2[1]:
                l1.append(k1)
                l2.append(k2[0])
                l3.append(k2[1])
                l4.append(v2)
    data_graph = pd.DataFrame(list(zip(l1, l2, l3, l4)),
                              columns=['t', 'u', 'v', 'w'])

    # Weights
    # Scale the weights of connection [0-1]
    X = [[entry] for entry in data_graph['w']]
    if save_weights: np.savetxt(file_output[8], X, delimiter=',', fmt='%s')

    # Plot the distribution of original weights
    plt.figure()
    # ax = sns.distplot(X, bins=10)
    ax = sns.distplot(X,
                      bins=max(X),
                      kde=True,
                      hist_kws={
                          "linewidth": 15,
                          'alpha': 1
                      })
    ax.set(xlabel='Distribution', ylabel='Frequency')

    # Max-Min Normalizer (produce many zeros)
    # transformer = MinMaxScaler()
    # X_scaled = transformer.fit_transform(X)
    # Returning column to row vector again
    # X_scaled = [entry[0] for entry in X_scaled]

    # Quantile normalizer (normal distribution)
    # transformer = QuantileTransformer()
    # Quantile normalizer (uniform distribution)
    transformer = QuantileTransformer(n_quantiles=1000,
                                      output_distribution='uniform')
    X_scaled = transformer.fit_transform(X)
    X_scaled = [entry[0] for entry in X_scaled]

    # Normalize by dividing to max
    # X_max = max(data_graph['w'])
    # X_scaled = [entry[0] / X_max for entry in X]

    # Fixing 0's and 1's entries
    # X_scaled = [entry if entry != 1 else 0.99 for entry in X_scaled]
    # X_scaled = [entry if entry > 0 else 0.1 for entry in X_scaled]
    
    # Scale everything between [a,b] or [0.5,1]
    # Because we do not want these weight become less than temporal weights
    # X_scaled = (b - a) * ((X_scaled - min(X_scaled)) / (max(X_scaled) - min(X_scaled))) + a
    X_scaled = (0.5 * np.array(X_scaled)) + 0.5
    
    # Rounding to X decimal point
    # X_scaled = [round(entry, 2) for entry in X_scaled]  # List
    X_scaled = np.round(X_scaled, 2)  # Array

    # Plot the distribution of scaled weights
    plt.figure()
    # ax = sns.distplot(X_scaled, bins=10)
    ax = sns.distplot(X_scaled, bins=10, kde=True, hist_kws={'alpha': 1})
    ax.set(xlabel='Distribution ', ylabel='Frequency')

    # Save scaled weights back to DF
    data_graph['w'] = X_scaled

    # Save the new weights
    if save_weights:
        np.savetxt(file_output[10], X_scaled, delimiter=',', fmt='%s')

    # Save to DB
    if save_network_db:
        data_graph[['u', 'v', 't', 'w']].to_sql(name='bluetooth_edgelist',
                                                con=db_connect(file_input[1]),
                                                if_exists='replace',
                                                index_label='id')

    # Save to file
    if save_network_csv:
        data_graph[['u', 'v', 't', 'w']].to_csv(file_output[2],
                                                header=False,
                                                index=False)

    # Add edges to network object
    # Complete network object
    for row in data_graph.itertuples(index=True, name='Pandas'):
        graph.add_edge(getattr(row, 'u'),
                       getattr(row, 'v'),
                       t=getattr(row, 't'),
                       w=getattr(row, 'w'))

    # Save graph
    if save_network_file:
        nx.write_gpickle(graph, file_output[0])

    # Save timestamps
    if save_times:
        np.savetxt(file_output[4] + '.csv', times, delimiter=',', fmt='%s')
        # pd.DataFrame(sorted(list(times))).to_csv(file_output[4], header=None, index=False)
        times_set = set()
        for u, v, w in graph.edges(data=True):
            times_set.add(w['t'])
        times = pd.Series(sorted(list(times_set)))
        np.savetxt(file_output[4], times, delimiter=',', fmt='%s')

    # Save nodes
    if save_nodes:
        # List of nodes in the graph
        nodes = pd.Series(sorted(list(graph.nodes)))
        # Save node list in a file "node.csv"
        np.savetxt(file_output[6], nodes, delimiter=',', fmt='%s')
        # pd.DataFrame(sorted(list(times))).to_csv(file_output[6], header=None, index=False)

    # Bipartite network
    # Complete network object
    l1, l2, l3, l4 = [], [], [], []
    for k1, v1 in time_bipartite_edges.items():
        for k2, v2 in v1.items():  # k2 = edge = (u,v)
            if k2[0] != k2[1]:
                l1.append(k1)
                l2.append(k2[0])
                l3.append(k2[1])
                l4.append(v2)
    data_bi_graph = pd.DataFrame(list(zip(l1, l2, l3, l4)),
                                 columns=['t', 'u', 'v', 'w'])

    # Weights
    X = [[entry] for entry in data_bi_graph['w']]
    if save_weights: np.savetxt(file_output[9], X, delimiter=',', fmt='%s')
    transformer = QuantileTransformer(n_quantiles=100,
                                      output_distribution='uniform')
    X_scaled = transformer.fit_transform(X)
    X_scaled = [entry[0] for entry in X_scaled]
    if save_weights:
        np.savetxt(file_output[11], X_scaled, delimiter=',', fmt='%s')
    # data_bi_graph['w'] = X_scaled

    # Save to DB
    if save_network_db:
        data_bi_graph[['u', 'v', 't',
                       'w']].to_sql(name='bluetooth_bipartite_edgelist',
                                    con=db_connect(file_input[1]),
                                    if_exists='replace',
                                    index_label='id')

    # Save to file
    if save_network_csv:
        data_bi_graph[['u', 'v', 't', 'w']].to_csv(file_output[3],
                                                   header=False,
                                                   index=False)

    # Add nodes and then edges
    # We need to add a prefix "u_" for users & "b_" for BT devices to the node id
    # So that we can distinguish them from each others
    for row in data_bi_graph.itertuples(index=True, name='Pandas'):
        # In bluetooth connections, user devices ID are repeated in all BT devices
        # So there is no need to differentiate between them, unless for some research necessary
        bipartite.add_edge(getattr(row, 'u'),
                           getattr(row, 'v'),
                           t=getattr(row, 't'),
                           w=getattr(row, 'w'))
        # Uncomment next 5 lines if wanna difrentiate users from other devices
        # node_user = 'u_' + str(getattr(row, 'u'))
        # node_ap = 'b_' + str(getattr(row, 'v'))
        # bipartite.add_node(node_user, bipartite=0)
        # bipartite.add_node(node_ap, bipartite=1)
        # bipartite.add_edge(node_user, node_ap, t=getattr(row, 't'), w=getattr(row, 'w'))

    # Save graph
    if save_network_file:
        nx.write_gpickle(bipartite, file_output[1])

    # Save timestamps
    if save_times:
        times_set = set()
        for u, v, w in bipartite.edges(data=True):
            times_set.add(w['t'])
        times_bipartite = pd.Series(sorted(list(times_set)))
        np.savetxt(file_output[5], times_bipartite, delimiter=',', fmt='%s')

    # Save nodes
    if save_nodes:
        # List of nodes in the bipartite graph
        nodes = pd.Series(sorted(list(bipartite.nodes)))
        # Save node list in a file "node.csv"
        np.savetxt(file_output[7], nodes, delimiter=',', fmt='%s')
        # pd.DataFrame(sorted(list(times))).to_csv(file_input[2], header=None, index=False)

    # Print network statistics
    if output_network:
        print('Temporal netwrok:')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        print('T =', len(times))
        print('---')
        print('Bipartite Temporal netwrok:')
        print('N =', bipartite.number_of_nodes())
        print('L =', bipartite.number_of_edges())
        print('T =', len(times_bipartite))

    return graph
    # return graph, bipartite

## Read

In [5]:
def temporal_bt_read(
    file_input=['network/bt_temporal_network.gpickle'],
    label_input='',
    output=False,
):
    """
    Reads temporal graph
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Network
    graph = nx.MultiDiGraph()
    # Read from file
    if os.path.exists(file_input[0]):
        if output: print('Reading temporal network ...')
        graph = nx.read_gpickle(file_input[0])
    else:
        if output: print('Temporal network file was not found')
        return None
    # Print graph statistics
    if output: print(nx.info(graph))
    return graph


def temporal_bt_times_read(
    file_input=['network/bt_temporal_times.csv'],
    label_input='',
    output=False,
):
    """
    Reads timestamps of temporal graph
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Times
    times = []
    # Read from file
    if os.path.exists(file_input[0]):
        if output: print('Reading times ...')
        times = pd.read_csv(
            file_input[0], index_col=False, header=None, names=['times']
        ).iloc[:, 0]
    else:
        if output: print('Times file was not found')
        return None
    # Change type (str -> datetime)
    times = times.astype('datetime64[ns]')
    return times


def temporal_bt_nodes_read(
    file_input=['network/bt_temporal_nodes.csv'],
    label_input='',
    output=False,
):
    """
    Reads nodes of temporal graph
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Nodes
    nodes = []
    # Read from file
    if os.path.exists(file_input[0]):
        if output: print('Reading nodes ...')
        nodes = pd.read_csv(
            file_input[0], index_col=False, header=None, names=['nodes']
        ).iloc[:, 0]
    else:
        if output: print('Nodes file was not found')
        return None
    return nodes

## Test

In [ ]:
# T = temporal_bt_create()

In [ ]:
# T = temporal_bt_create(
#     output_times=False,
#     output_network=True,
#     save_times=False,
#     save_nodes=False,
#     save_weights=False,
#     save_network_db=False,
#     save_network_csv=False,
#     save_network_file=False
# )

In [ ]:
# T = temporal_bt_read(output_network=True)

## Result

For now, we are normalizing the link weight over all of them possible weight (not focusing on time of the connection or the end nodes of a pair in the connection)    
But a future goal is to normalize on the both of time dimension and the pair-wise nodes ... Therefore we decide to not to normalize in aggregate temporal network step, rather do this later over the Directed-time-series network creation step

# Static Network

## Build

In [ ]:
def static_bt_create(
    file_input=[
        'network/bt_temporal_network.gpickle', 'network/bt_temporal_times.csv'
    ],
    file_output=[
        'network/bt_static_network.gpickle', 'network/bt_static_edgelist.csv'
    ],
    input_label='',
    output_label='',
    temporal=None,
    undirected=True,
    output_network=False,
    save_network_csv=False,
    save_network_file=False,
):
    """
    Convert input temporal network to the (aggregated) static network
    the edge weights are sum of temporal interactions over entire time window
    """

    # Modify input & output file names
    file_input = label_amend(file_input, input_label)
    file_output = label_amend(file_output, output_label)

    # Read temporal network from file
    times = []
    if temporal is None:
        temporal, times = temporal_bt_read(
            file_input=[file_input[0], file_input[1]])

    # Create empty static network
    graph = nx.Graph()

    # Update static network edges
    for u, v, data in temporal.edges(data=True):
        t = 1 if 't' in data else 0
        if graph.has_edge(u, v):
            graph[u][v]['w'] += t
            graph[u][v]['s'] = graph[u][v]['s'] + data['w']
        else:
            graph.add_edge(u, v, w=t, s=data['w'])

    # Devide weights (both requency and strength) by 2, because they have been counted twice
    if input_undirected:
        for u, v, data in graph.edges(data=True):
            graph[u][v]['w'] //= 2
            graph[u][v]['s'] //= 2

    # Print network statistics
    if output_network:
        print('static netwrok:')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        if nx.algorithms.components.is_connected(graph):
            print('Network is connected with diameter',
                  nx.algorithms.distance_measures.diameter(graph))
        else:
            print('Network is not connected and has {} connected components.'.
                  format(
                      nx.algorithms.components.number_connected_components(
                          graph)))
        print('Density =', nx.classes.function.density(graph))

    # Save the network
    if save_network_file:
        nx.write_gpickle(graph, file_output[0])

    if save_network_csv:
        # (1)
        # nx.write_edgelist(graph, file_output[1], data=True)
        # (2)
        # List of list containing columns of [[node-1],[node-2],[weight-1(e.g. time)],[weight-2],...]
        w_keys = list(
            list(graph.edges(list(T.nodes(0))[0][0], data=True))[0][2].keys())
        edgelist_columns = []
        edgelist_columns.append([])
        edgelist_columns.append([])
        for w_id in range(len(w_keys)):
            edgelist_columns.append([])
        for u, v, w in graph.edges(data=True):
            edgelist_columns[0].append(u)
            edgelist_columns[1].append(v)
            for i in range(len(w_keys)):
                edgelist_columns[2 + i].append(w[w_keys[i]])
            # The associative undirected edge
            edgelist_columns[1].append(u)
            edgelist_columns[0].append(v)
            for i in range(len(w_keys)):
                edgelist_columns[2 + i].append(w[w_keys[i]])
        pd.DataFrame(edgelist_columns).transpose().to_csv(file_output[1],
                                                          header=False,
                                                          index=False)

    return graph

## Read

In [ ]:
def static_bt_read(file_input=['network/bt_static_network.gpickle'],
                input_label='',
                output_network=False):

    # Modify input file names
    file_input = label_amend(file_input, input_label)

    # Read the network from file
    graph = nx.Graph()
    print('Reading the static network ...')
    graph = nx.read_gpickle(file_input)

    # Print network statistics
    if output_network:
        print('static netwrok:')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        if nx.algorithms.components.is_connected(graph):
            print('Network is connected with diameter',
                  nx.algorithms.distance_measures.diameter(graph))
        else:
            print('Network is not connected and has {} connected components.'.format(
                nx.algorithms.components.number_connected_components(graph)))
        print('Density =', nx.classes.function.density(graph))

    return graph

## Run

## Result

# Time-ordered Network

## Create
TN network for the Bluetooth connections

#### __Light__ version
Skip creating edge weights

In [6]:
def tn_bt_light_create(file_input=[
    'db/uiuc.db', 'network/bt_temporal_network.gpickle',
    'network/bt_temporal_times.csv'
],
                       file_output=[
                           'network/bt_tn_light_network.gpickle',
                           'network/bt_tn_light_edgelist.csv'
                       ],
                       temporal=None,
                       times=None,
                       label_input='',
                       label_output='',
                       _directed=True,
                       _trans=True,
                       _teleport=False,
                       _loop=False,
                       output_network=False,
                       save_network_db=False,
                       save_network_csv=True,
                       save_network_file=True):
    """
    Convert a temporal (aggregated) network to a (directed) time-ordered (temporal) network
    Light version means edges are not labelled
        _directed: add bi-directional temporal edges i.e. t <-> t+1
        _teleport: activate temporal teleportation
        _loop: add last timestamp nodes to first i.e. T -> t1
    """
    # Amend input / output file names using label
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read temporal networks and its node list from file
    if temporal is None or times is None:
        temporal, times = temporal_bt_read(
            file_input=[file_input[1], file_input[2]])

    # GRAPH
    graph = nx.DiGraph()

    # TIME
    # Size of timestamp list
    T = len(times)
    # Time -> Index
    time_index = dict((v, i) for i, v in enumerate(times))

    # NODE
    # Size of node list
    N = temporal.number_of_nodes()
    # Node -> Index
    nodes = pd.Series(sorted(list(temporal.nodes)))
    node_index = dict((v, i) for i, v in enumerate(nodes))

    # EDGE
    # Node (index) -> horizontal edges
    L = temporal.number_of_edges()
    node_edges = {}
    for n in range(N):
        node_edges[n] = [(N * t + n, N * (t + 1) + n) for t in range(T)]

    # Horizontal edges
    if _directed:
        for node, edges in node_edges.items():
            for i in range(len(edges)):  # [0,T]
                # Add edges: node(i) -> node(i+1)
                graph.add_edge(edges[i][0], edges[i][1])
                # With temporal teleportation
                if _teleport:
                    for j in range(i + 1, len(edges)):
                        graph.add_edge(edges[i][0], edges[j][1])
        # With temporal loop
        if _loop:
            for node in node_edges:
                graph.add_edge(node_edges[node][-1][1], node_edges[node][0][0])
    else:  # Undirected
        for node, edges in node_edges.items():
            for i in range(len(edges)):
                graph.add_edge(edges[i][0], edges[i][1])
                # Backward horizontal edge (i.e. moving back in time)
                graph.add_edge(edges[i][1], edges[i][0])
                if _teleport:
                    for j in range(i + 1, len(edges)):
                        graph.add_edge(edges[i][0], edges[j][1])
                        # Backward teleportation in time
                        graph.add_edge(edges[j][1], edges[i][0])
        # With tempora loop
        if _loop:
            for node in node_edges:
                graph.add_edge(node_edges[node][-1][1], node_edges[node][0][0])
                graph.add_edge(node_edges[node][0][0], node_edges[node][-1][1])

    # Crossed edges
    # Only edge weight carrying to lite version is 'weight' of interactions
    # Becasue we need 'weight' for spread probability edge attribute
    if _directed:  # Directed
        for u, v, edge_data in temporal.edges(data=True):
            u_index = node_index[u]
            v_index = node_index[v]
            t_index = time_index[edge_data['t']]
            graph.add_edge(u_index + t_index * N,
                           v_index + (t_index + 1) * N,
                           w=edge_data['w'])
    else:  # Undirected
        for u, v, edge_data in temporal.edges(data=True):
            u_index = node_index[u]
            v_index = node_index[v]
            t_index = time_index[edge_data['t']]
            graph.add_edge(u_index + t_index * N,
                           v_index + (t_index + 1) * N,
                           w=edge_data['w'])
            graph.add_edge(v_index + (t_index + 1) * N,
                           u_index + t_index * N,
                           w=edge_data['w'])

    # Transitive closure
    trans_num = 0
    if _trans:
        for t in range(T):
            snap_nodes = [(t * N) + n for n in range(N)]
            snap_nodes.extend([((t + 1) * N) + n for n in range(N)])
            snap_graph = graph.subgraph(snap_nodes)
            A = nx.to_numpy_matrix(snap_graph)
            A_t = A[:len(A) // 2, len(A) // 2:]
            snap_trans = nx.to_numpy_matrix(
                nx.transitive_closure(
                    nx.from_numpy_matrix(A_t, create_using=nx.DiGraph)))
            # Compare edges of transitive closure with edges we had before, find new edges, add them to network
            snap_edges = np.transpose(np.nonzero(A_t != snap_trans))
            snap_weights = np.tile(
                0.5 * np.random.sample(len(snap_edges) // 2) + 0.5, 2)
            # index of new edges should be converted into node ID in network
            for r in range(len(snap_edges)):
                if not graph.has_edge(snap_nodes[snap_edges[r][0]],
                                      snap_nodes[snap_edges[r][1] + N]):
                    trans_num += 1  # Counter of transitive edges
                    graph.add_edge(snap_nodes[snap_edges[r][0]],
                                   snap_nodes[snap_edges[r][1] + N],
                                   w=snap_weights[r],
                                   trans=True)
                    if not _directed:
                        graph.add_edge(snap_nodes[snap_edges[r][0]] + N,
                                       snap_nodes[snap_edges[r][1]],
                                       w=snap_weights[r],
                                       trans=True)

    # Print
    if output_network:
        print('Time-ordered Network (Light)')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        if _trans:
            print('{} transitive closure edges were added.'.format(trans_num))

    # Save
    if save_network_db:
        edge_list = pd.DataFrame.from_dict(graph.edges)
        edge_list.columns = ['u', 'v']
        edge_list.to_sql(name='bluetooth_time_ordered_edgelist',
                         con=db_connect(file_input[0]),
                         if_exists='replace',
                         index_label='id')

    if save_network_file:
        nx.write_gpickle(graph, file_output[0])

    if save_network_csv:
        # nx.write_edgelist(graph, file_output[1])
        nx.write_weighted_edgelist(graph, file_output[1], delimiter=',')

    return graph

#### __Full__ version
TS network for the Bluetooth connections

In [7]:
def tn_bt_full_create(file_input=[
    'db/uiuc.db', 'network/bt_temporal_network.gpickle',
    'network/bt_temporal_times.csv', 'network/bt_temporal_nodes.csv'
],
                      file_output=[
                          'network/bt_tn_full_network.gpickle',
                          'network/bt_tn_full_edgelist.csv',
                          'network/bt_tn_delta.csv',
                          'network/bt_tn_weights.csv'
                      ],
                      temporal=None,
                      times=None,
                      label_input='',
                      label_output='',
                      _directed=True,
                      _trans=True,
                      _teleport=False,
                      _loop=False,
                      _color=True,
                      _delta=True,
                      _delta_type='h',
                      version=0,
                      _omega=1,
                      _gamma=0.0001,
                      _epsilon=1,
                      _distance=1,
                      _alpha=0.5,
                      output_delta=False,
                      output_weight=False,
                      output_network=False,
                      save_delta=False,
                      save_weight=False,
                      save_network_csv=True,
                      save_network_file=True):
    """
    Convert a temporal (aggregated) network to a (directed) time-series (DS) temporal network
    Full version means nodes and edges get labelled with differnt information such as:
        * color
        * position
        * delta time: the time differnec of an edge from previous timestamp
        * temporal weight considering horizontal and crossed format with following versions:
            0: None
            1: (1/2)^(lengh of horizontal path)
            2: 1/(lenght of horizontal path)
            3: 1/(log2(lenght of horizontal path))
    Other parameters are:
        _directed: add bi-directional temporal edges i.e. t <-> t+1
        _teleport: activate temporal teleportation
        _loop: add last timestamp nodes to first i.e. T -> t1
        _color: add color to nodes
        _delta: add delta-time weight to edges
        _delta_type: h (for hour) & m (for minute)
        _omega: horizontal edge weight (by default = 1, without penalize)
        _gamma: teleportation edge weight
        _epsilon: crossed edge weight
        _distance: horizontal edge distance
        _alpha: dynamic penalizing coefficient
        _sigma: crossed teleportation (not used here - applied during HITS)
    """
    # Amend input / output file names using label
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read temporal networks and its node list from file
    if temporal is None or times is None:
        temporal, times = temporal_bt_read(
            file_input=[file_input[1], file_input[2]])

    # GRAPH
    graph = nx.DiGraph()

    # TIME
    # Size of timestamp list
    T = len(times)
    # Time -> Index
    time_index = dict((v, i) for i, v in enumerate(times))

    # NODE
    # Size of node list
    N = temporal.number_of_nodes()
    # Node -> Index
    nodes = pd.Series(sorted(list(temporal.nodes)))
    node_index = dict((v, i) for i, v in enumerate(nodes))

    # EDGE
    # Node (index) -> horizontal edges
    L = temporal.number_of_edges()
    node_edges = {}
    for n in range(N):
        node_edges[n] = [(N * t + n, N * (t + 1) + n) for t in range(T)]

    # Colors for nodes at differnt timestamp
    colors = []
    if _color:
        cmap = cm.get_cmap('Wistia', T + 1)
        for i in range(cmap.N):
            rgb = cmap(i)[:3]
            colors.append(matplotlib.colors.rgb2hex(rgb))
    else:
        colors = ['#000000'] * (T + 1)

    # Create the time difference between all consecutive timestamps
    # Convert delta to second and then hour
    # Add '1' to the begginig so that -> len(delta) == len(times)
    # Because there is nothing before the first timestamp
    # So we assume delta for nodes at the first timestamp is '1' hour
    delta = []
    if _delta:
        times = list(times)
        delta_temp = pd.Series(pd.Series(times[1:]) - pd.Series(times[:T - 1]))
        if _delta_type == 'h':  # Hour scale
            delta = [int(ts.total_seconds() // 3600) for ts in delta_temp]
        elif _delta_type == 'm':  # Minute scale
            delta = [int(ts.total_seconds() // 60) for ts in delta_temp]
        delta.insert(0, 1)  # (postion, value)
        times = pd.Series(times)
    else:
        delta = [1] * T  # 1 hour (default value for all edges)
        times = pd.Series(times)

    if output_delta:
        # Count the unique delta values
        print("Delta time distribution:")
        if _delta_type == 'h':
            delta_count = pd.Series(Counter(delta)).sort_index()
            # print(delta_count[:24])  # Print deltas up to 1 day = 24 hours
            print(delta_count)
        elif _delta_type == 'm':
            # 1 day = 24 H * 60 Min = 1440, anything more than that could be outlier
            # Or some sort of time jumps in the dataset
            delta_count = pd.Series([d for d in delta if d <= 24 * 60
                                     ]).value_counts(normalize=True)
            print(delta_count)

    if save_delta:  # save delta
        np.savetxt(file_output[2], delta, delimiter=',', fmt='%s')
        # pd.DataFrame(delta).to_csv(file_output[2], header=None, index=False)

    # Convert delta list to series for easy access in future
    delta = pd.Series(delta)

    # Horizontal edges
    if _directed:
        for node, edges in node_edges.items():
            # Add the first node at the first timestamp
            graph.add_node(
                node,
                # It turned out 'parent' is not necessary, because we use 'node' or parent_index in 'pos'
                # So it can always be extracted from postion of node or id/number_of_nodes
                # parent=nodes[node],  # Parent
                c=colors[0],  # Color
                p=(0, node))  # Position / Cordinates
            for i in range(len(edges)):  # i = time, and in range [0,T]
                # Add the edge (u,v) with its attribute
                graph.add_edge(
                    edges[i][0],
                    edges[i][1],
                    t=times[i],  # Time
                    d=delta[i],  # Delta or temporal distance
                    tw=_omega,  # Temporal weight (tw)
                    c='silver')  # Color

                # Then set node attribute of second node from created edge of (u,v)
                # graph.nodes[edges[i][1]]['parent'] = nodes[node]
                graph.nodes[edges[i][1]]['c'] = colors[i + 1]
                graph.nodes[edges[i][1]]['p'] = (i + 1, node)
                # With temporal teleportation
                if _teleport:
                    for j in range(i + 1, len(edges)):
                        graph.add_edge(
                            edges[i][0],
                            edges[j][1],
                            # d=sum(delta[i:j])  # Needs testing
                            tw=_gamma,
                            c='gold')
        # With temporal loop
        if _loop:
            for node in node_edges:
                graph.add_edge(
                    node_edges[node][-1][1],
                    node_edges[node][0][0],
                    # d=sum(delta)  # Needs testing
                    tw=_omega,
                    c='orange')
    else:  # Undirected
        for node, edges in node_edges.items():
            graph.add_node(node, c=colors[0], p=(0, node))
            for i in range(len(edges)):
                graph.add_edge(edges[i][0],
                               edges[i][1],
                               t=times[i],
                               d=delta[i],
                               tw=_omega,
                               c='silver')
                # Backward horizontal edge (i.e. moving back in time)
                graph.add_edge(edges[i][1],
                               edges[i][0],
                               t=times[i],
                               d=delta[i],
                               tw=_omega,
                               c='silver')
                graph.nodes[edges[i][1]]['c'] = colors[i + 1]
                graph.nodes[edges[i][1]]['p'] = (i + 1, node)
                if _teleport:
                    for j in range(i + 1, len(edges)):
                        graph.add_edge(edges[i][0],
                                       edges[j][1],
                                       tw=_gamma,
                                       c='gold')
                        # Backward teleportation in time
                        graph.add_edge(edges[j][1],
                                       edges[i][0],
                                       tw=_gamma,
                                       c='gold')
        # With temporal loop
        if _loop:
            for node in node_edges:
                graph.add_edge(node_edges[node][-1][1],
                               node_edges[node][0][0],
                               tw=_omega,
                               c='orange')
                graph.add_edge(node_edges[node][0][0],
                               node_edges[node][-1][1],
                               tw=_omega,
                               c='orange')

    # Crossed edges
    if _directed:
        for u, v, edge_data in temporal.edges(data=True):
            u_index = node_index[u]
            v_index = node_index[v]
            t_index = time_index[edge_data['t']]
            graph.add_edge(u_index + t_index * N,
                           v_index + (t_index + 1) * N,
                           t=edge_data['t'],
                           w=edge_data['w'],
                           d=delta[t_index],
                           tw=_epsilon,
                           c='black')
    else:  # Undirected
        for u, v, edge_data in temporal.edges(data=True):
            u_index = node_index[u]
            v_index = node_index[v]
            t_index = time_index[edge_data['t']]
            graph.add_edge(u_index + t_index * N,
                           v_index + (t_index + 1) * N,
                           t=edge_data['t'],
                           w=edge_data['w'],
                           d=delta[t_index],
                           tw=_epsilon,
                           c='black')
            graph.add_edge(v_index + (t_index + 1) * N,
                           u_index + t_index * N,
                           t=edge_data['t'],
                           w=edge_data['w'],
                           d=delta[t_index],
                           tw=_epsilon,
                           c='black')

    # Transitive closure
    trans_num = 0
    if _trans:
        for t in range(T):
            snap_nodes = [(t * N) + n for n in range(N)]
            snap_nodes.extend([((t + 1) * N) + n for n in range(N)])
            snap_graph = graph.subgraph(snap_nodes)
            A = nx.to_numpy_matrix(snap_graph)
            A_t = A[:len(A) // 2, len(A) // 2:]
            snap_trans = nx.to_numpy_matrix(
                nx.transitive_closure(
                    nx.from_numpy_matrix(A_t, create_using=nx.DiGraph)))
            # Compare edges of transitive closure with edges we had before, find new edges, add them to network
            snap_edges = np.transpose(np.nonzero(A_t != snap_trans))
            snap_weights = np.tile(
                0.5 * np.random.sample(len(snap_edges) // 2) + 0.5, 2)
            # index of new edges should be converted into node ID in network
            for r in range(len(snap_edges)):
                if not graph.has_edge(snap_nodes[snap_edges[r][0]],
                                      snap_nodes[snap_edges[r][1] + N]):
                    trans_num += 1  # Counter of transitive edges
                    graph.add_edge(snap_nodes[snap_edges[r][0]],
                                   snap_nodes[snap_edges[r][1] + N],
                                   t=times[t],
                                   w=snap_weights[r],
                                   d=delta[t],
                                   tw=_epsilon,
                                   trans=True,
                                   c='red')  # Only for trans edges
                    if not _directed:
                        graph.add_edge(snap_nodes[snap_edges[r][0]] + N,
                                       snap_nodes[snap_edges[r][1]],
                                       t=times[t],
                                       w=snap_weights[r],
                                       d=delta[t],
                                       tw=_epsilon,
                                       trans=True,
                                       c='red')

    # Prepare for penalizing horizontal edge weights
    # If version == 0 -> skip penalizing all together
    if version != 0:
        for node, edges in node_edges.items():
            for i in range(len(edges)):  # len = T
                # If in-degree second node 'v' of underlying edge (u,v) is 1
                # Meaning it (u,v) is a horizontal edge with no other node connecting to it
                if graph.in_degree(edges[i][1]) == 1:  # 'v' of (u,v)
                    if i == 0:  # There is no edge before so ...
                        graph[edges[i][0]][edges[i][1]][
                            'tw'] = 1 + _distance  # e.g. _distance = 1
                    else:
                        graph[edges[i][0]][edges[i][1]]['tw'] = graph[edges[
                            i - 1][0]][edges[i - 1][1]]['tw'] + _distance

    # (alpha)^(distance) e.g. (1/2)^(1), (1/2)^(2), ...
    # This version is exponential penalization = very harsh
    if version == 1:
        for node, edges in node_edges.items():
            for i in range(len(edges)):
                graph[edges[i][0]][edges[i][1]]['tw'] = _alpha**(
                    graph[edges[i][0]][edges[i][1]]['tw'] - 1)

    # 1/(distance)
    # This version is polinomial penalization = harsh :/
    elif version == 2:
        for node, edges in node_edges.items():
            for i in range(len(edges)):
                graph[edges[i][0]][edges[i][1]]['tw'] = 1 / graph[edges[i][0]][
                    edges[i][1]]['tw']

    # 1/log2(distance + 1)
    # This version is logarithmic penalization = not so harsh :)
    elif version == 3:
        for node, edges in node_edges.items():
            for i in range(len(edges)):
                graph[edges[i][0]][edges[i][1]]['tw'] = 1 / np.log2(
                    graph[edges[i][0]][edges[i][1]]['tw'] + 1)

    if save_weight:
        ew = {}
        for u, v, tw in graph.edges(data='tw'):
            ew[(u, v)] = tw
        pd.DataFrame.from_dict(ew, orient='index').to_csv(file_output[3])

    # Print temporal weights for horizontal edges
    if output_weight:
        print('node @ time -> edge(u,v) = weight')
        for node, edges in node_edges.items():
            for edge in edges:
                print('{} @ {}: ({},{}) = {}'.format(
                    node, edge[1] // N, edge[0], edge[1],
                    graph[edge[0]][edge[1]]['tw']))

    if output_network:
        print('Time-ordered Network (Full):')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        if _trans:
            print('{} transitive closure edges were added.'.format(trans_num))

    if save_network_file:
        nx.write_gpickle(graph, file_output[0])

    if save_network_csv:
        w_keys = list(
            list(graph.edges(list(graph.nodes(0))[0][0],
                             data=True))[0][2].keys())
        # list of list containing columns of [[node-1],[node-2],[weight-1(e.g. time)],[weight-2],...]
        edgelist_columns = []
        edgelist_columns.append([])
        edgelist_columns.append([])
        for w_id in range(len(w_keys)):
            edgelist_columns.append([])
        for u, v, w in graph.edges(data=True):
            edgelist_columns[0].append(u)
            edgelist_columns[1].append(v)
            for i in range(len(w_keys)):
                edgelist_columns[2 + i].append(w[w_keys[i]])
        pd.DataFrame(edgelist_columns).transpose().to_csv(file_output[1],
                                                          header=False,
                                                          index=False)

    return graph

## Reverse (TODO)
Convert a (multi-layer) time-ordered network to (multi-edge) temporal network

In [ ]:
def tn_bt_to_temporal(file_input=[
    'remove/bt_tn_network.gpickle', 'network/bt_temporal_times.csv'
],
                      file_output=[
                          'remove/bt_temporal_network.gpickle',
                          'remove/bt_temporal_edgelist.csv',
                          'remove/bt_temporal_times.csv',
                          'remove/bt_temporal_nodes.csv'
                      ],
                      temporal=None,
                      label_input='',
                      label_output='',
                      output_network=False,
                      save_network_csv=True,
                      save_network_file=True):
    """
    Convert a (directed) time-ordered network (TON) to a (multi-edge) temporal (T) network
    """
    # Amend filenames
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Create empty TEMPORAL network
    graph = nx.MultiDiGraph()

    # Read temporal networks from file
    if temporal is None:
        temporal = nx.read_gpickle(file_input[0])

    # Timestamp
    # times = temporal_bt_times_read()  # TODO: we may not need it
    T = line_count(file_input[2])
    
    for 


## Read

In [8]:
def tn_bt_light_read(
    file_input=['network/bt_tn_light_network.gpickle'],
    label_input='',
    output=False
):
    """
    Reads light version of (directed) time-ordered network of Bluetooth connections
    """
    file_input = label_amend(file_input, label_input)
    graph = nx.read_gpickle(file_input[0])
    graph.name = 'Time-ordered Network'
    if output: print(nx.info(graph))
    return graph


def tn_bt_full_read(
    file_input=['network/bt_tn_full_network.gpickle'],
    label_input='',
    output=False
):
    """
    Reads full version of (directed) time-ordered network of Bluetooth connections
    """
    file_input = label_amend(file_input, label_input)
    graph = nx.read_gpickle(file_input[0])
    graph.name = 'Time-ordered Network (Full Version)'
    if output: print(nx.info(graph))
    return graph

## Analyze

In [ ]:
# ! pip install mpmath
# ! pip install powerlaw

import powerlaw

def tn_bt_analyze(file_input=[
    'network/bt_tn_full_network.gpickle',
    'network/bt_temporal_network.gpickle', 'network/bt_temporal_times.csv'
],
                  label_input=''):
    # Read network
    graph = dt_read(file_input=[file_input[0]], label_input=label_input)
    temporal, times = temporal_bt_read(
        file_input=[file_input[1], file_input[2]], output_network=True)

    # Variable
    T = len(times)
    N = temporal.number_of_nodes()
    L = temporal.number_of_edges()
    # N_new = graph.number_of_nodes()
    # L_new = graph.number_of_edges()

    # Dictionary of time -> all nodes in that time
    time_nodes = {}
    for t in range(T):
        time_nodes[t] = [N * t + n for n in range(N)]

    # Check the edge frequency in each timestamp
    time_out_degrees = {}
    for t in sorted(time_nodes):  # t in [0,T]
        time_out_degrees[t] = [graph.out_degree(n) - 1 for n in time_nodes[t]]
    # Dataframe of outdegress with time as columns
    time_out_degrees = pd.DataFrame.from_dict(time_out_degrees)
    # out_degrees_time = time_out_degrees.sum(0)  # Sum of out degrees over column i.e. time
    print(sorted(Counter(time_out_degrees.sum(0)).items(), key=lambda x: x[0]))
    # results = powerlaw.Fit(out_degrees_time)
    # print(results.power_law.alpha)
    # print(results.power_law.xmin)
    # R, p = results.distribution_compare('power_law', 'lognormal')
    # print(out_degrees_time / max(out_degrees_time))  # Scale of out_degrees_time to [0,1] 
    print(time_out_degrees.sum(1))  # Sum of out degrees over row i.e. node

In [ ]:
# tn_bt_analyze()

## Run

In [ ]:
# tn = tn_bt_light_create(
#     _directed=True,
#     _teleport=False,
#     _trans=True,  # transitive edges
#     _loop=False,
#     output_network=True,
#     save_network_db=True,
#     save_network_csv=True,
#     save_network_file=True)

In [ ]:
# tn = tn_bt_full_create(
#     _directed=True,
#     _trans=True,
#     _teleport=False,
#     _loop=False,
#     _color=True,
#     _delta=True,
#     version=3,  # 0
#     _omega=1,  # 1
#     _gamma=0.0001,
#     _epsilon=1,
#     _distance=0.1,  # 1
#     _alpha=0.5,
#     output_delta=False,
#     output_weight=False,
#     output_network=True,
#     save_delta=True,
#     save_weight=False,
#     save_network_csv=True,
#     save_network_file=True)

In [ ]:
tn = tn_bt_light_read()

In [ ]:
# tn = tn_bt_full_read()

# Edge Weight

## Create
We set the temporal edge weights (from light version of time-ordered network) can be save in a new dictionary to keep the network light or save into network     
In addition, add a new weight __probs__ or __normalized_weights__ which can be used in spread / diffusion process

#### Edge weight

In [9]:
def ew_create(
    file_input=[
        'network/bt_tn_light_network.gpickle',
        'network/bt_temporal_nodes.csv',
        'network/bt_temporal_times.csv',
    ],
    file_output=['network/bt_tn_weights.csv'],
    label_input='',
    label_output='',
    graph=None,
    number_of_nodes=None,
    number_of_times=None,
    version=0,
    omega=1,
    epsilon=1,
    gamma=0.0001,
    distance=1,
    alpha=0.5,
    save_weights=False,
    output_weights=False,
    plot_weights=False
):
    """
    Calculate (and save) the temporal edge weights
    
    Parameters
    ----------
    graph : NetworkX
        time-ordered network (TON)
    number_of_nodes : int
        numbero of nodes from the temporal graph (default is None, then reads from file)
    number_of_times : int
        numbero of timestamps from the temporal graph (default is None, then reads from file)
    version : int
        0 -> contact value of omega
        1 -> dynamic (alpha)^(lengh_of_horizontal_path)
        2 -> dynamic (1)/(lenght_of_horizontal_path)
        3 -> dynamic (1)/(log2(lenght_of_horizontal_path))
    omega : float
        weight factor of horizontal edges (e.g. 1, 0.01, 0.001, 0.0001 ...)
    epsilon :float
        weight factor of crossed edgess (e.g. 1, 0.01, 0.001, 0.0001 ...)
    gamma : float
        weight of horizontal teleport edges (e.g. 0.0001, 0.001, 0.01 ...)
    distance : float
        value that being added to as lenght to non-active consecutive edges or paths (smaller -> slower weight decay)
    alpha : float
        magnification factor in version 1 (larger -> slower weight decay), default = 1/2 or 0.5
    
    Returns
    -------
    dict
        {(u,v):temporal_weight}
    """
    def has_crossed_edge(in_graph, in_N, in_node):
        in_parent = in_node % in_N
        for pre in in_graph.predecessors(in_node):
            if pre % in_N != in_parent:
                return True
        # Else = no crossed edge found ...
        return False

    # Amend input / output file names using label
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read time-ordered network and size of node from original network
    if graph is None:
        graph = tn_bt_light_read(file_input=[file_input[0]])

    if number_of_nodes is None:
        N = line_count(file_input[1])
    else:
        N = number_of_nodes

    if number_of_times is None:
        T = line_count(file_input[2])
    else:
        T = number_of_times
        # T = graph.number_of_nodes() // N

    # Edge-Weight dictionary
    # {(u,v):weight}
    ew = {}

    # Horizontal edges helper dictionary
    hedges = {}

    # If node in_degree = 1 then node 'u' has only '1' horizontal-edge of (v,u)
    nid = 1

    _directed = True
    # If node 0 at time 1 connects to node 0 at time 0 -> undirected
    if (1 * N) + 0 in graph.predecessors(0):
        print('Graph is temporally undirected')
        _directed = False
        nid += 2

    _teleport = False
    # If node 0 at time 0 connects to node 0 at time 2 -> teleport
    if (2 * N) + 0 in graph.successors(0):
        print('Graph has temporal teleporation edges')
        _teleport = True

    _loop = False
    # If node 0 at time T connects to node 0 at time 0 -> loop
    if (T * N) + 0 in graph.predecessors(0):
        print('Graph has temporal loop edges')
        _loop = True

    if version == 0:  # Without penalization
        if output_weights:
            print(
                f'Penalizing horizontal edges with contact value of omega = {omega}'
            )
        for u, v in graph.edges():
            # When u & v have same parent -> edge (u,v) is horizontal
            time_delta = abs(v - u) // N
            parent_u = u % N
            parent_v = v % N
            if parent_u != parent_v:  # Crossed
                ew[(u, v)] = epsilon  # E.g. 1
            else:
                if time_delta > 1:  # Teleport
                    ew[(u, v)] = gamma  # E.g. 0.0001
                else:  # # Horizontal OR time_delta = 1
                    # Node v is node u at one timestamp after
                    ew[(u, v)] = omega

    else:  # Penalize ...
        # Nodes [0-N]
        for u, v in sorted(graph.edges(), key=lambda x: x[0]):
            time_delta = abs(v - u) // N
            parent_u = u % N
            parent_v = v % N
            if parent_u != parent_v:
                ew[(u, v)] = epsilon
            else:
                if time_delta > 1:
                    ew[(u, v)] = gamma
                else:
                    # Node v has crossed edge
                    # if graph.in_degree(v) != nid:  # 1 or 2
                    if has_crossed_edge(graph, N, v):
                        hedges[(u, v)] = omega  # E.g. 1
                    else:
                        # Node v does not have crossed edge
                        # Look at the previous edge weight (if exsit, otherwise return omega)
                        hedges[(u, v)
                               ] = hedges.get((u - N, u), omega) + distance

        # Update weights based on version of penalization
        if version == 1:
            # Decay exponentially fast
            # (parameteralpha)^(distance) e.g. 1/2^1 , 1/2^2, ...
            for edge, weight in hedges.items():
                hedges[edge] = alpha**(weight - 1)
        elif version == 2:
            # Decay very fast
            # 1/(distance) e.g. 1/2, 1/3, ...
            for edge, weight in hedges.items():
                hedges[edge] = 1 / weight
        elif version == 3:
            # Decay fast
            # 1/log2(distance + 1) e.g. 1/log2, 1/log3, ...
            for edge, weight in hedges.items():
                hedges[edge] = 1 / np.log2(weight + 1)

        # Finish by updating helper dictionary to 'ew'
        ew.update(hedges)

    if save_weights:
        # dict_save(ew, file_output[0][:-4])  # npy
        # OR
        # dict_save(ew, file_output[0], method='c', sort=True)  # csv
        # OR
        # pd.DataFrame.from_dict(ew, orient='index').to_csv(file_output[0], header=False)
        # OR
        pd.Series(ew).reset_index().to_csv(
            file_output[0], header=False, index=False
        )

    if plot_weights and version > 0:
        ls = sorted(ew.items())
        ls1, ls2 = zip(*ls)
        plt.figure()
        ax = sns.displot(ls2, kind='kde')

    if output_weights:
        for e, w in sorted(ew.items(), key=lambda x: x[0]):
            if e[0] % N == e[1] % N:  # H
                if graph.in_degree(e[1]) == nid:
                    print('{}\t->\t{}\t{}'.format(e[0], e[1], w))

    return ew

#### Transition probability

In [ ]:
def ew_prob_create(
    file_input=[
        'network/bt_tn_light_network.gpickle',
        'network/bt_temporal_nodes.csv',
        'network/bt_temporal_times.csv',
    ],
    file_output=[
        'network/bt_tn_weights.csv',
        'network/bt_tn_probs.csv',
    ],
    label_input='',
    label_output='',
    graph=None,
    number_of_nodes=None,
    number_of_times=None,
    _version=0,
    _omega=1,
    _gamma=0.0001,
    _epsilon=1,
    _distance=1,
    _alpha=0.5,
    save_weights=True,
    save_probs=True,
    output_weights=True,
    output_probs=True
):
    """
    {(u,v):temporal_weight}
    {(u,v):probability}
    Calculate and set the edge weight (save in dictionary / into graph)
    Also, create edge probability for spread (according to weights)
    Versions:
        0: None (or fixed value of omega)
        1: (1/2)^(lengh of horizontal path)
        2: 1/(lenght of horizontal path)
        3: 1/(log2(lenght of horizontal path))
    Parameters:
        * graph (networkx): directed temporal network i.e. DT
        * undirected (bool): True, if 'dt' is temporaly bi-directional
        * omega (float): horizontal edge weight - only used when penalize=False (e.g. 1 ,10 ,100, 1000 ...)
        * epsilon (float): crossed edge weight (e.g. 0.1, 1 ,10 ,100 ...)
        * gamma (float): horizontal edge teleport weight (e.g. 0.0001, 0.001, 0.01 ...)
        * penalize (bool): True, if want to weight horizontal-edges dynamically
        * distance (float): penalizing (V1,V2,V3) weight - smaller => slower decay
        * alpha (float): penalizing (only in V1)
    Notes:
        * when we don't penalize the horizontal edges
            set a fix value for weights of epsilon and omega (e.g. 1 & 1 or 1 & 10, ...)
        * When we penalize
            larger alpha => slower decay in weights
            larger distance => faster decay in weights
            find a balance between 'alpha' and 'distance' based on number of consecutive horizontals
    """
    def has_crossed_edge(in_graph, in_N, in_node):
        in_parent = in_node % in_N
        for pre in in_graph.predecessors(in_node):
            if pre % in_N != in_parent:
                return True
        # Else = no crossed edge found ...
        return False

    # Amend input / output file names using label
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read time-ordered network and size of node from original network
    if graph is None:
        graph = tn_bt_light_read()

    if number_of_nodes is None:
        N = line_count(file_input[1])
    else:
        N = number_of_nodes

    if number_of_times is None:
        T = line_count(file_input[2])
    else:
        T = number_of_times
        # T = graph.number_of_nodes() // N

    # Edge-Weight dictionary
    # {(u,v):weight}
    ew = {}

    # Horizontal edges helper dictionary
    hedges = {}

    # If node in_degree = 1 then node 'u' has only '1' horizontal-edge of (v,u)
    nid = 1

    _directed = True
    # If node 0 at time 1 connects to node 0 at time 0 -> undirected
    if (1 * N) + 0 in graph.predecessors(0):
        _directed = False
        nid += 2

    _teleport = False
    # If node 0 at time 0 connects to node 0 at time 2 -> teleport
    if (2 * N) + 0 in graph.successors(0):
        _teleport = True

    _loop = False
    # If node 0 at time T connects to node 0 at time 0 -> loop
    if (T * N) + 0 in graph.predecessors(0):
        _loop = True

    if _version == 0:  # Without penalization
        for u, v in graph.edges():
            # u & v have same parent => (u,v) is horizontal
            time_delta = abs(v - u) // N
            parent_u = u % N
            parent_v = v % N
            if parent_u != parent_v:  # Crossed
                ew[(u, v)] = _epsilon  # E.g. 1
            else:  # Horizontal
                if time_delta > 1:  # Teleport
                    # If _teleport = False during TN, these edges do not exist
                    ew[(u, v)] = _gamma  # E.g. 0.0001
                else:  # time_delta = 1
                    # Node v is node u at one timestamp after
                    ew[(u, v)] = _omega

    else:  # Penalize ...
        # Nodes [0-N]
        for u, v in sorted(graph.edges(), key=lambda x: x[0]):
            time_delta = abs(v - u) // N
            parent_u = u % N
            parent_v = v % N
            if parent_u != parent_v:
                ew[(u, v)] = _epsilon
            else:
                if time_delta > 1:
                    ew[(u, v)] = _gamma
                else:
                    # Node v has crossed edge
                    # if graph.in_degree(v) != nid:  # 1 or 2
                    if has_crossed_edge(graph, N, v):
                        hedges[(u, v)] = _omega  # E.g. 1
                    else:
                        # Node v does not have crossed edge
                        # Look at the previous edge weight (if exsit, otherwise return omega)
                        hedges[(u, v)
                               ] = hedges.get((u - N, u), _omega) + _distance

        # Update weights based on version of penalization
        if _version == 1:
            # Decay exponentially fast
            # (parameter_alpha)^(distance) e.g. 1/2^1 , 1/2^2, ...
            for edge, weight in hedges.items():
                hedges[edge] = _alpha**(weight - 1)
        elif _version == 2:
            # Decay very fast
            # 1/(distance) e.g. 1/2, 1/3, ...
            for edge, weight in hedges.items():
                hedges[edge] = 1 / weight
        elif _version == 3:
            # Decay fast
            # 1/log2(distance + 1) e.g. 1/log2, 1/log3, ...
            for edge, weight in hedges.items():
                hedges[edge] = 1 / np.log2(weight + 1)

        # Finish by updating helper dictionary to 'ew'
        ew.update(hedges)

    if save_weights:
        # dict_save(ew, file_output[0][:-4])  # npy
        # dict_save(ew, file_output[0], method='c', sort=True)  # csv
        pd.DataFrame.from_dict(ew, orient='index').to_csv(file_output[0])
        if _version > 0:
            ls = sorted(ew.items())
            ls1, ls2 = zip(*ls)
            plt.figure()
            ax = sns.distplot(
                ls2,
                kde=True,
            )

    if output_weights:
        for e, w in sorted(ew.items(), key=lambda x: x[0]):
            if e[0] % N == e[1] % N:  # H
                if graph.in_degree(e[1]) == nid:
                    print('{}\t->\t{}\t{}'.format(e[0], e[1], w))

    # Normalize the weight of node each nodes out-links over Max() or Sum()
    # To have spread probability of each link
    # First, we scale horizontal edges to [0-0.5] and crossed ones

    # Edge-Probability dictionary
    # {(u,v):probability}
    # prob = {}
    prob = ew.copy()

    for u, v, data in graph.edges(data=True):
        # u & v have same parent => (u,v) is horizontal
        time_delta = abs(v - u) // N
        parent_u = u % N
        parent_v = v % N
        if parent_u != parent_v:  # Crossed
            ew[(u, v)] = _epsilon  # E.g. 1
        else:  # Horizontal
            if time_delta > 1:  # Teleport
                # If _teleport = False during TN, these edges do not exist
                ew[(u, v)] = _gamma  # E.g. 0.0001
            else:  # time_delta = 1
                # Node v is node u at one timestamp after
                ew[(u, v)] = _omega

    # X_scaled = (b - a) * ((X_scaled - min(X_scaled)) / (max(X_scaled) - min(X_scaled))) + a
    # X_scaled = (0.5 * np.array(X_scaled)) + 0.5

    for n in graph:
        parent_n = n % N
        w_c = []  # Weights of crossed edges
        w_h = []  # Weights of horizontal edges
        for s in graph.successors(n):
            parent_s = s % N
            if parent_n != parent_s:  # Crossed
                # We read the weight of edge coming from aggregated network
                w_c.append(graph[n][s]['w'])
            else:  # Horizontal or teleport ...
                w_h.append(prob[(n, s)])
        # If more than just one horizontal or crossed
        if len(w_c) + len(w_h) > 1:
            w_c_m = 0
            if len(w_c) > 0:
                w_c_m = max(w_c)
            w_h_m = 0
            if len(w_h) > 0:
                w_h_m = max(w_h)
            # Adjust weights of horizontal according to maximum of crossed
            if w_h_m > w_c_m:
                w_h = [item if item < w_c_m else w_c_m for item in w_h]
            # Update probabilities
            for s in graph.successors(n):
                parent_s = s % N
                if parent_n != parent_s:
                    # Option 1
                    # Leave them to original value from temporal network e.g. [0.5,1]
                    # Most likely many are 0.5 which have 50/50 chance of transmission
                    prob[(n, s)] = graph[n][s]['w']
                    # Option 2
                    # Scale up so maximum is 1.0 while rest are relatively scaled up too
                    # prob[(n, s)] = graph[n][s]['w'] / w_c_m
                else:
                    if prob[(n, s)] < w_c_m:
                        # Option 1
                        # Leave it as it is
                        # prob[(n, s)] = prob[(n, s)]
                        pass
                        # Option 2
                        # Normalized it with maximum value
                        # prob[(n, s)] /= w_c_m
                    else:
                        # Means horizontal weight is larger than max(crossed)
                        # I.e. If exist a C(rossed) < 1 => exist H(orizontal) = 1
                        # Option 1
                        # Scale down 'H' to max('C') and end up being normalized to 1.0
                        # prob[(n, s)] = 1.0
                        # Option 2
                        # Scale down to a random value [0.5,1]
                        prob[(n, s)] = 0.5 * np.random.sample() + 0.5

    if save_probs:
        pd.DataFrame.from_dict(prob, orient='index').to_csv(file_output[1])

    # Check the distribution of probabilities
    if output_probs:
        ls = sorted(prob.items())
        ls1, ls2 = zip(*ls)
        plt.figure()
        ax = sns.distplot(ls2, kde=True)
        # ax = sns.distplot([i for i in ls2 if i != 1], kde=True)

    return ew, prob

## Read

In [10]:
def ew_read(file_input=['network/bt_tn_weights.csv'], label_input=''):
    """
    Read edge weights of time-ordered graph
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Read edge weights file
    ew = pd.read_csv(file_input[0], names=['u', 'v', 'w'])
    # Fix index from tuple of (u,v)
    ew.index = list(zip(ew.u, ew.v))
    # Only keep the weights
    ew = ew['w']
    # Convert to dict
    ew = ew.to_dict()
    return ew


def prob_read(file_input=['network/bt_tn_probs.csv'], label_input=''): # TODO
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Read edge transmission probability file
    prob = pd.read_csv(file_input[1], index_col=0)
    prob = prob.to_dict()['0']
    prob = {eval(k): float(v) for k, v in prob.items()}
    # prob = dict_load(file_input[1][:-4])

    return prob

## Test

#### Constant Penalize

In [ ]:
# ew = ew_create(
#     version=0,
#     omega=0.5,
#     epsilon=1,
#     gamma=0.0001,
#     distance=1,
#     alpha=0.5,
#     save_weights=False,
#     output_weights=False
# )

#### Dynamic Penalize

In [ ]:
# Current config
# ew = ew_create(
#     label_output='',
#     version=3,
#     omega=1,
#     epsilon=1,
#     gamma=0.0001,
#     distance=0.1,
#     alpha=0.5,
#     save_weights=True,
#     output_weights=False,
#     plot_weights=False
# )

In [ ]:
# New config
# This shows more uniform dist for values != 1 and less penalize magnitude compared to omega = 1.0 
# ew = ew_create(
#     label_output='',
#     version=3,
#     omega=1.1,
#     epsilon=1,
#     gamma=0.0001,
#     distance=0.1,
#     alpha=0.5,
#     save_weights=False,
#     output_weights=False,
#     plot_weights=True
# )

#### Read

In [ ]:
# ew = ew_read()

# Spread (TODO: Must complete ...)

Spread Model

In [ ]:
def spread(graph, initial_nodes=None, initial_size=1, time_stop=100):
    """
    Spread
    """

    N = graph.number_of_nodes()
    # nodes = graph.nodes  # TODO: can be removed
    time_passed = 0
    # Susceptible = 0
    # Infected = 1
    # Recovered = 2
    # Blocked = -1
    # Every node is susecptible at first
    status = {n: 0 for n in graph.nodes}
    # Randomly select (one or more) patient zero(s)
    if initial_nodes is None:
        # Fraction infected ratio
        if initial_size > 0 and initial_size < 1:
            initial_size = int(initial_size * N)
            # Case when ratio is too low compared to N
            if initial_size == 0:
                initial_size = 1
        # Specific number of infected nodes (e.g. 1)
        initial_nodes = np.random.choice(graph.nodes,
                                         size=initial_size,
                                         replace=False)
    # Update initial infected and remaining susecptible
    for n in initial_nodes:
        status[n] = 1
    i_nodes = set(initial_nodes)
    s_nodes = set([n for n in status if status[n] == 0])

# Influence Maximization

**Linear Threshold Model (LTM):**    
In this model, each node has a random threshold __theta_v__ in range [0,1]. For simplicity, we can set all the nodes to an specific threshold of __theta__.    
Also, ledges of each node is associated with a weight and summation of the wights <= 1, basically, if the summation of these links weighs > threshold => activate the node    
    
**Independent Cascade Model (ICM):**    
Unlike LTM, in this model, nodes do not have threshold. Se when a node 'v' become active it activate neighbors
***
In out temporal network, the link has a temporal weight or __tp__. We can (L1) normalize the tp's so for each node, so SUM of wights = 1
In other words, we turn tp weights to __p__ weights required for __probabilistic contagion__ or linear threshold model.

## Main

In [ ]:
def influence_maximization(graph,
                           n_nodes=1,
                           probability=0.2,
                           n_iters=1000,
                           output_log=False,
                           output_result=True):
    """
    Parameters:
        n_nodes: number of desired nodes to include in influential set
        p: probability of walking to neighbor in range [0,1]
        n_iters: number of iteration of different cascase using a given set of nodes
    Return:
        influential_nodes: set of most influential node with size < n_nodes
        max_spread: maximum of average spread for each node in the set
    Note:
        - we find the set of influential nodes but we don't know whos is the most influential one
        unless we run the set from size 1 to k in a progressive approach and rank the node in a list
        - in temporal network if node is in earlier timestamp it become more influential because it reaches more
        and to avoid this we can define a time windows with fix size or loop the end time to start time
    """
    max_spreads = []
    node_spreads = []
    progress_spreads = []
    influential_nodes = []
    N = graph.number_of_nodes()
    for n_idx in range(n_nodes):
        best_node = -1
        best_spread = -np.inf
        remaining_nodes = graph.nodes - influential_nodes
        # Look into set of not influential nodes and find a new candidates
        for node in remaining_nodes:
            candidate_nodes = influential_nodes + [node]
            spread = []
            for i in range(n_iters):
                new_active, visited = candidate_nodes[:], candidate_nodes[:]
                while new_active:
                    # Get neighbours of newly activated node
                    targets = []
                    for n in new_active:
                        targets += graph.neighbors(n)
                    # Calculatie if any of those neighbours can be activated, if yes add them to new_ones
                    np.random.seed(i)
                    success = np.random.uniform(0, 1,
                                                len(targets)) < probability
                    # new_ones = list(np.extract(success, sorted(targets)))
                    new_ones = list(np.extract(success, targets))
                    # Checking which ones in new_ones are not in visited
                    # Only adding them to our visited so that no duplicate in visited
                    # Basically, this is independent cascade model where a node can not get infeccted more than once
                    new_active = list(set(new_ones) - set(visited))
                    visited += new_active
                # Increase number of overal visited or spread
                spread.append(len(visited))
            # Average the spread over number of iterations
            avg_spread = np.mean(spread)
            # Log
            if output_log:
                print('Candidate node {} has average spread {}'.format(
                    node, avg_spread))
                print('Candidate list:', candidate_nodes)
            # If candidate node is better than best node -> replace them
            if avg_spread > best_spread:
                progress_spreads.append((node, avg_spread))
                best_spread = avg_spread
                best_node = node
        # Add the best node and its average spread to the answer
        influential_nodes.append(best_node)
        if n_idx == 0:
            node_spreads.append(best_spread)
        else:
            node_spreads.append(best_spread - max_spreads[n_idx - 1])
        max_spreads.append(best_spread)

    if output_log:
        print('Influential nodes and their spreads:')
        print(progress_spreads)

    if output_result:
        print('Influence Maximization\n---')
        print('N =', graph.number_of_nodes())
        print('L =', graph.number_of_edges())
        print(
            '---\n{} of the most influential nodes with highest spread\naveraged over {} iterations'
            .format(n_nodes, n_iters))
        dash = '-' * 60
        print(dash)
        print(
            '{:<8}{:^8}{:^20}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:>8}'.format(
                'Rank', 'Node', 'Node', 'Node', 'Total', 'Spread', 'In', 'Out',
                'C', 'B', 'Page'))
        print(
            '{:<8}{:^8}{:^20}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:>8}'.format(
                '', 'Index', 'Name', 'Spread', 'Spread', 'Ratio (%)', 'Degree',
                'Degree', 'Crt', 'Crt', 'Crt'))
        print(dash)

        in_degree_sequence = sorted([(d, n) for n, d in graph.in_degree()],
                                    reverse=True)
        out_degree_sequence = sorted([(d, n) for n, d in graph.out_degree()],
                                     reverse=True)
        in_degree_max = in_degree_sequence[0][0]
        out_degree_max = in_degree_sequence[0][0]
        cc = nx.closeness_centrality(graph)
        cc_sorted = sorted(cc.items(), key=lambda x: x[1], reverse=True)
        cc_max = cc_sorted[0][1]
        bt = nx.betweenness_centrality(graph)
        bt_sorted = sorted(bt.items(), key=lambda x: x[1], reverse=True)
        bt_max = bt_sorted[0][1]
        pg = nx.pagerank(graph)
        pg_sorted = sorted(pg.items(), key=lambda x: x[1], reverse=True)
        pg_max = pg_sorted[0][1]

        for i in range(len(influential_nodes)):
            print('{:<8}{:^8}{:^20}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:^8}{:>8}'.
                  format(i, graph.nodes[influential_nodes[i]]['idx'],
                         influential_nodes[i], round(node_spreads[i],
                                                     2), max_spreads[i],
                         round(max_spreads[i] / N * 100, 2),
                         graph.in_degree(influential_nodes[i]),
                         graph.out_degree(influential_nodes[i]),
                         round(cc[influential_nodes[i]]/cc_max, 2),
                         round(bt[influential_nodes[i]]/bt_max, 2),
                         round(pg[influential_nodes[i]]/pg_max, 2)))


#         print(*influential_nodes)
#         print(*max_spreads)

    return influential_nodes, max_spreads

## Test

In [ ]:
g = nx.read_gpickle('file/sleeping_giant_directed.gpickle')

In [ ]:
inf, sp = influence_maximization(g,
                                 n_nodes=10,
                                 probability=0.5,
                                 n_iters=100,
                                 output_log=False,
                                 output_result=True)

In [ ]:
inf

# HITS

## Create

In [179]:
def hits(
    file_input=[
        'network/bt_tn_full_network.gpickle',
        'network/bt_tn_light_network.gpickle',
        'network/bt_temporal_times.csv',
        'network/bt_temporal_nodes.csv',
        'network/bt_tn_weights.csv',
    ],
    file_output=[
        'hits/a.csv',
        'hits/h.csv',
    ],
    label_input='',
    label_output='',
    graph=None,
    times=None,
    nodes=None,
    ew=None,
    nstart=None,
    full=False,
    version=0,
    sigma=0.85,
    max_iter=100,
    tol=1.0e-8,
    norm_max=True,
    norm_final_l1=True,
    norm_final_l2=False,
    norm_iter=False,
    norm_degree=False,
    norm_damping=False,
    output=False,
    plot=False,
    save=True
):
    """
    Calculate HITS centrality of time-ordered network (TON)
    
    Parameters
    ----------
    version : int
        (1) NetworkX
            Normalize scores with SUM=1 Range [0,1]
            1/max normalization in each iteration and final normalization at the end
        (2) Book
            Normalize scores with SUM=1 and DEVIATION=1 and range=[0,1]
        (3) Paper (randomization or teleportation)
            No regular normalization (v1 & v2), just in-out-normalization & damping/teleport/randomize
        (4) NetX + teleport or PARTIAL randomization (NO in-out-normalization)
        (5) Book + teleport or PARTIAL randomization (NO in-out-normalization)
        (6) Paper + normalization of scores at each iteration
        (7) Paper + l2 normalization of score at the end
        (0) Default -> no change to parameters (norm_*) -> version 1 (NetX) + other manually set parameters
    
    Returns
    -------
    dict , dict
        authority and hub scores as {node:score}
    """

    # Edit filenames
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Finishing iteration
    finish_iter = max_iter

    h = {}  # Hub scores
    a = {}  # Authority scores

    # Read graph
    if graph == None:
        if full:  # Full version
            graph = tn_bt_full_read(file_input=[file_input[0]])
        else:  # Light version
            graph = tn_bt_light_read(file_input=[file_input[1]])

    # Number of nodes in time-ordered network (TON)
    N_t = graph.number_of_nodes()

    # Times
    if times is None:
        times = list(temporal_bt_times_read([file_input[2]]))
        T = line_count(file_input[2])
    else:
        T = len(times)

    # Nodes
    if nodes is None:
        nodes = list(temporal_bt_nodes_read([file_input[3]]))
        N = line_count(file_input[3])
    else:
        N = len(nodes)
        # N = graph.number_of_nodes() // T

    # Edge-weight
    if ew is None: ew = ew_read(file_input=[file_input[4]])

    # Damping-coefficient or normalization-parameter (default = 0.85) is same as sigma in supra-matrix model which applies to crossed edges only
    # But here applies to all of the edges (crossed and horizontals ...)
    damping = 1 - sigma  # Default = 0.15

    # NetX
    if version == 1:
        norm_max = True  #
        norm_final_l1 = True  #
        norm_final_l2 = False
        norm_iter = False
        norm_degree = False
        norm_damping = False

    # Book
    if version == 2:
        norm_max = False
        norm_final_l1 = False
        norm_final_l2 = False
        norm_iter = True  #
        norm_degree = False
        norm_damping = False

    # Paper
    if version == 3:
        norm_max = False
        norm_final_l1 = False
        norm_final_l2 = False
        norm_iter = False
        norm_degree = True  #
        norm_damping = True  #

    # NetX + teleport
    if version == 4:
        norm_max = True  #
        norm_final_l1 = True  #
        norm_final_l2 = False
        norm_iter = False
        norm_degree = False
        norm_damping = True  #

    # Book + teleport
    if version == 5:
        norm_max = False
        norm_final_l1 = False
        norm_final_l2 = False
        norm_iter = True  #
        norm_degree = False
        norm_damping = True  #

    # Paper (degree normalization + teleport) + NetX
    if version == 6:
        norm_max = True  #
        norm_final_l1 = True  #
        norm_final_l2 = False
        norm_iter = False
        norm_degree = True  #
        norm_damping = True  #

    # Paper (degree normalization + teleport) + Book
    if version == 7:
        norm_max = False
        norm_final_l1 = False
        norm_final_l2 = False
        norm_iter = True  #
        norm_degree = True  #
        norm_damping = True  #

    # Paper + final norm L2
    if version == 8:
        norm_max = False
        norm_final_l1 = False
        norm_final_l2 = True  #
        norm_iter = False
        norm_degree = True  #
        norm_damping = True  #

    # Paper + final norm L2
    if version == 9:
        norm_max = True  #
        norm_final_l1 = False
        norm_final_l2 = False
        norm_iter = False
        norm_degree = True  #
        norm_damping = True  #

    # Initialize scores
    if nstart is None:
        h = dict.fromkeys(graph, 1.0 / graph.number_of_nodes())
    else:
        h = nstart
        # Normalize starting vector
        s = 1.0 / sum(h.values())
        for k in h:
            h[k] *= s

    # Power iteration
    if output: print(f'Calculating HITS (Version {version}) ...')
    # Set start time
    start_time = timeit.default_timer()

    for i in range(max_iter):

        if i != 0 and i % 10 == 0:
            elapsed = timeit.default_timer() - start_time
            if output: print(f'Iteration {i} completed {elapsed:.0f} seconds')
            # Reset start time
            start_time = timeit.default_timer()

        # Save the last calculated hub score
        hlast = h

        # Initialize scores for new iteration
        h = dict.fromkeys(hlast.keys(), 0)
        a = dict.fromkeys(hlast.keys(), 0)

        # Authority
        # ---------
        # Left multiply a^T=hlast^T*G
        # Authority of neighbors get value from hub of current node, influenced by weight of edge

        # (1) default HITS -> no degree normalization
        if not norm_degree:
            for n in h:
                for nbr in graph[n]:
                    a[nbr] += hlast[n] * ew.get((n, nbr), 1)
        # (2) degree normalization
        else:
            for n in h:
                for nbr in graph[n]:
                    a[nbr] += hlast[n] / graph.in_degree(nbr) * ew.get(
                        (n, nbr), 1
                    )

        # Paper
        # Apply damping factor OR random-walk OR teleport
        if norm_damping:
            for n in a:
                a[n] *= sigma
                a[n] += damping

        # Book
        # Normalized authority scores over root of sum of squares after each calculation
        # In this case we dont need final normalization
        if norm_iter:
            s = 1.0 / math.sqrt(sum([x**2 for x in a.values()]))
            for n in a:
                a[n] *= s

        # Hub
        # ---
        # Multiply h=Ga
        # Hub of current node get value from authority values of neighbor nodes, influenced by weight of edge

        # (1) default HITS -> no degree normalization
        if not norm_degree:
            for n in h:
                for nbr in graph[n]:
                    h[n] += a[nbr] * ew.get((n, nbr), 1)
        # (2) degree normalization
        else:
            for n in h:
                for nbr in graph[n]:
                    h[n] += a[nbr] / graph.out_degree(n) * ew.get((n, nbr), 1)

        # Paper
        # Apply damping factor OR randomize
        if norm_damping:
            for n in h:
                h[n] *= sigma
                h[n] += damping

        # Book
        # Normalized hub scores over root of sum of squares after each calculation
        # In this case we dont need final normalization
        if norm_iter:
            s = 1.0 / math.sqrt(sum([x**2 for x in h.values()]))
            for n in h:
                h[n] *= s

        # END of one iteration

        # NetX
        # Normalize scores over maximum
        # Stopping score from getting really large
        if norm_max:
            a_max = 1.0 / max(a.values())
            h_max = 1.0 / max(h.values())
            for n in a:  # OR for n in h => both are same
                a[n] *= a_max
                h[n] *= h_max

        # Check convergence
        err = sum([abs(h[n] - hlast[n]) for n in h])
        if err < tol:
            finish_iter = i
            if output: print(f'Successful after {finish_iter} iteration')
            break

    # Program did not meet treshhold for convergence
    if finish_iter == max_iter:
        if output: print(f'Not converged after {finish_iter} iterations')

    # NetX
    # Last normalization (L1) using sum
    # Output in range (0-1) with sum-all = 1
    if norm_final_l1:
        a_sum = 1.0 / sum(list(a.values()))
        h_sum = 1.0 / sum(list(h.values()))
        for n in a:  # OR for n in h => both are same
            a[n] *= a_sum
            h[n] *= h_sum

    # Last normalization (L2) using sum squred
    if norm_final_l2 and not norm_final_l1:
        a_sum = 1.0 / math.sqrt(sum([x**2 for x in a.values()]))
        h_sum = 1.0 / math.sqrt(sum([x**2 for x in h.values()]))
        for n in a:  # OR for n in h => both are same
            a[n] *= a_sum
            h[n] *= h_sum

    # Save
    if save:
        pd.DataFrame.from_dict(a, orient='index'
                               ).to_csv(file_output[0], header=False)
        pd.DataFrame.from_dict(h, orient='index'
                               ).to_csv(file_output[1], header=False)

    # Plot distribution of scores
    if plot:
        # In most cases, A & H has same value so ploting one of them is enough
        ls_a = sorted(a.values())  # ls_h = sorted(h.values())
        ax = sns.displot(ls_a, kde=True, rug=True)
        # BUT if they were different, we can create a dataframe and plot both together
        # df = pd.DataFrame([a,h]).T
        # df.columns = 'a h'.split()
        # plt.figure()
        # ax = sns.displot(data=df, kind='kde')

    return a, h

## Conditional

In [284]:
def hits_conditional(
    file_input=['hits/a.csv', 'hits/h.csv'],
    file_output=[
        'hits/a_array.csv',
        'hits/h_array.csv',
        'hits/a_avg_node.csv',
        'hits/a_avg_time.csv',
        'hits/h_avg_node.csv',
        'hits/h_avg_time.csv',
        'hits/a_norm_node.csv',
        'hits/a_norm_time.csv',
        'hits/h_norm_node.csv',
        'hits/h_norm_time.csv',
    ],
    label_input='',
    label_output='',
    a=None,
    h=None,
    N=None,
    T=None,
    removed=False,
    save=True
):
    """
    Create conditional centralities by normalizing or averaging HITS scores over time (column) or node (row)
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Read scores from file
    if a is None or h is None:
        a, h = hits_read(file_input=file_input)

    # Times
    if T is None:
        times = list(temporal_bt_times_read())
        T = len(times)

    # Nodes
    if N is None:
        nodes = list(temporal_bt_nodes_read())
        N = len(nodes)

    # Convert score dict to matrix (N x T)
    A = np.zeros((N, T + 1))
    H = np.zeros((N, T + 1))
    # If graph is complete (no node has been removed)
    if not removed:
        a_sorted = [a[k] for k in sorted(a.keys())]
        h_sorted = [h[k] for k in sorted(h.keys())]
        A = np.reshape(a_sorted, (T + 1, N)).T
        H = np.reshape(h_sorted, (T + 1, N)).T
    else:  # Some of the nodes were removed
        for node in a.keys():  # h.keys()
            row = node % N  # node
            col = node // N  # time
            A[row, col] = a[node]
            H[row, col] = h[node]

    # Save A & H matrices
    if save:
        pd.DataFrame(A).to_csv(file_output[0], header=None, index=None)
        pd.DataFrame(H).to_csv(file_output[1], header=None, index=None)

    # Read A & H matrices
    # A = pd.read_csv(file_input[0]).values
    # H = pd.read_csv(file_input[1]).values

    # Conditional Centralities
    # ------------------------

    # Average of score over time (column) or node (row)
    A_avg_node = np.mean(A, axis=1)
    A_avg_time = np.mean(A, axis=0)
    H_avg_node = np.mean(H, axis=1)
    H_avg_time = np.mean(H, axis=0)

    # Normalized scores over time (column) or node (row)
    # L1 is over sum(abs(x)) but scince all values are + L1 is over sum(x)
    A_norm_node = normalize(A, axis=1, norm='l1')
    A_norm_time = normalize(A, axis=0, norm='l1')
    H_norm_node = normalize(H, axis=1, norm='l1')
    H_norm_time = normalize(H, axis=0, norm='l1')
    # OR
    # A_norm_node = A / A.sum(axis=1, keepdims=True)
    # A_norm_time = A / A.sum(axis=0, keepdims=True)
    # H_norm_node = H / H.sum(axis=1, keepdims=True)
    # H_norm_time = H / H.sum(axis=0, keepdims=True)

    if save:
        pd.DataFrame(A_avg_node).to_csv(file_output[2], header=None, index=None)
        pd.DataFrame(A_avg_time).to_csv(file_output[3], header=None, index=None)
        pd.DataFrame(H_avg_node).to_csv(file_output[4], header=None, index=None)
        pd.DataFrame(H_avg_time).to_csv(file_output[5], header=None, index=None)
        pd.DataFrame(A_norm_node).to_csv(file_output[6], header=None, index=None)
        pd.DataFrame(A_norm_time).to_csv(file_output[7], header=None, index=None)
        pd.DataFrame(H_norm_node).to_csv(file_output[8], header=None, index=None)
        pd.DataFrame(H_norm_time).to_csv(file_output[9], header=None, index=None)

## Read

In [283]:
def hits_read(
    file_input=['hits/a.csv', 'hits/h.csv'],
    label_input='',
    output=False,
    plot=False
):
    """
    Read HITS centrality
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    # Read scores from file
    if output: print('Reading HITS score ...')
    a = pd.read_csv(file_input[0], names=['a'], index_col=0)
    h = pd.read_csv(file_input[1], names=['h'], index_col=0)
    # Take column a/h and convert to dict
    a = a.to_dict()['a']
    h = h.to_dict()['h']
    # Plot
    if plot: sns.displot(a.values(), kde=True, rug=True, legend=False)
    return a, h


def hits_conditional_read(
    file_input=[
        'hits/a_array.csv',
        'hits/h_array.csv',
        'hits/a_avg_node.csv',
        'hits/a_avg_time.csv',
        'hits/h_avg_node.csv',
        'hits/h_avg_time.csv',
        'hits/a_norm_node.csv',
        'hits/a_norm_time.csv',
        'hits/h_norm_node.csv',
        'hits/h_norm_time.csv',
    ],
    label_input='',
    return_matrices=[0, 1],
    output=False
):
    """
    Read conitional HITS centrality scores
    
    Parameters
    ----------
    return_matrices : list
        [0,1] -> return matrices A & H only
        range(0,10) -> return all conditional matrices
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)

    # Return object names
    rtn = {}
    rtn_names = [
        'A', 'H', 'a_avg_node', 'a_avg_time', 'h_avg_node', 'h_avg_time',
        'a_norm_node', 'a_norm_time', 'h_norm_node', 'h_norm_time'
    ]

    # Read scores from file
    if return_matrices == [0, 1]:  # Default
        if output: print('Reading HITS score matrices ...')
        A = pd.read_csv(file_input[0], header=None, index_col=False).values
        H = pd.read_csv(file_input[1], header=None, index_col=False).values
        return A, H
    else:
        if output: print('Readinging and returning ...')
        for i in return_matrices:
            if output: print(f'{rtn_names[i]}')
            temp = pd.read_csv(
                file_input[i], header=None, index_col=False
            ).values
            rtn[rtn_names[i]] = temp
    return rtn

## Analyze

In [281]:
def hits_analyze(
    file_input=[
        'network/bt_temporal_nodes.csv',
        'network/bt_temporal_times.csv',
    ],
    file_net=['network/bt_tn_light_network.gpickle'],
    file_ew=['network/bt_tn_weights.csv'],
    file_hits=[
        'hits/a.csv',
        'hits/h.csv',
        'hits/a_array.csv',
        'hits/h_array.csv',
        'hits/a_avg_node.csv',
        'hits/a_avg_time.csv',
        'hits/h_avg_node.csv',
        'hits/h_avg_time.csv',
        'hits/a_norm_node.csv',
        'hits/a_norm_time.csv',
        'hits/h_norm_node.csv',
        'hits/h_norm_time.csv',
    ],
    file_output=['hits/report.csv', 'hits/top.csv'],
    file_image=[
        'hits/fig_a.pdf',
        'hits/fig_h.pdf',
        'hits/fig_a_report.pdf',
        'hits/fig_h_report.pdf',
        'hits/fig_a_corr.pdf',
        'hits/fig_h_corr.pdf',
        'hits/fig_mat.pdf',
    ],
    label_input='',
    label_hits='',
    label_output='',
    label_image='',
    graph=None,
    times=None,
    nodes=None,
    ew=None,
    a=None,
    h=None,
    top=2,
    section=4,
    report_num=100
):
    """
    Analyze HITS scores
        Find top rank nodes using averaged-score over time
        Highlight top nodes of TON model and other info such as parent, time, in-out-degree, in-out-weight ...
    
    Parameters
    ----------
        a : dict
            authority score {node:score}
        h : dict
            hub score {node:score}
        graph: NetX
            time-ordered network (TON) model
        times : list
        nodes : list
        ew : dict
            edge weights {(u,v):w}
    """
    # Edit filenames
    file_input = label_amend(file_input, label_input)
    file_hits = label_amend(file_hits, label_hits)
    file_output = label_amend(file_output, label_output)
    file_image = label_amend(file_image, label_image)

    # Read scores from file
    if a is None or h is None:
        a, h = hits_read(file_input=file_hits[0:2])

    # Read graph
    if graph == None:
        graph = tn_bt_light_read(file_input=file_net)

    # Nodes
    if nodes is None:
        nodes = list(temporal_bt_nodes_read(file_input=[file_input[0]]))
    N = len(nodes)

    # Times
    if times is None:
        times = list(temporal_bt_times_read(file_input=[file_input[1]]))
    T = len(times)

    times.insert(0, times[0])
    t_first = times[0].strftime('%d %B\n%I %p')
    t_last = times[-1].strftime('%d %B\n%I %p')
    t_0 = times[0] - pd.Timedelta(1, unit='h')
    t_day_idx = []
    t_day_week = []
    t_day_date = []
    t_hour_12 = []
    t_hour_17 = []
    for i, time in enumerate(times):
        if t_0.weekday() != time.weekday():
            t_day_idx.append(i)
            t_day_week.append(time.strftime('%a'))
            t_day_date.append(time.strftime('%d'))
            t_0 = time
        if time.strftime('%H') == '12':
            t_hour_12.append(i)
        if time.strftime('%H') == '17':
            t_hour_17.append(i)

    # Edge-weight
    if ew is None: ew = ew_read(file_input=file_ew)

    # Conditional HTIS scores
    cs = hits_conditional_read(
        file_input=file_hits[2:12],
        label_input=label_hits,
        return_matrices=range(0, 10)
    )

    # Authority Plot
    # --------------

    fig, ax = plt.subplots(figsize=(32, 20))
    # (1)
    # norm = mpl.colors.Normalize(vmin=0, vmax=1.0, clip=True)
    # mapper = cm.ScalarMappable(norm=norm, cmap=cm.YlGnBu)
    # (2)
    # mapper = cm.get_cmap('YlGnBu', 10)  # 10 discrete colors
    # force the first color entry to be grey
    # (3)
    cmap = plt.cm.tab10_r  # Define the colormap (tab10_r, YlGnBu, Wistia, ocean, cool)
    # Extract all colors from the .jet map
    cmaplist = [cmap(i) for i in range(cmap.N)]
    # If tab10
    cmaplist[0], cmaplist[2] = cmaplist[2], cmaplist[0]
    # Force the first color entry to be grey
    cmaplist[0] = (.8, .8, .8, 1.0)
    # Create the new custom map
    mapper = mpl.colors.LinearSegmentedColormap.from_list(
        'my cmap', cmaplist, cmap.N
    )
    # define the bins and normalize
    bounds = np.linspace(0, 1, 11)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    # TODO: we can modify and add removed nodes or score == 0
    # as red dots or something like that in the plot, but we need to
    # know if score == 0 is considered as removed or we get a removed_list
    # of nodes as an input of the hits_analyze method

    A = cs['A']
    X = list(range(1, T + 2))
    for row in range(len(A)):
        sc = ax.scatter(
            X,
            [row + 1] * (T + 1),
            s=A[row] * 1000,
            # s=[4*x**2 for x in A[row] * 10],
            c=A[row],
            # c=[mapper.to_rgba(x) for x in A[row]],
            marker='|',
            alpha=0.8,
            # vmin=0,
            # vmax=1.0,
            norm=norm,  # or use vmin/vmax
            # cmap=cm.YlGnBu,
            cmap=mapper,
        )

    A_avg_node = cs['a_avg_node']
    ax.scatter(
        [T + 3] * N,
        list(range(1, N + 1)),
        s=A_avg_node * 100,
        c=A_avg_node,
        marker='s',
        alpha=0.5,
        cmap=cm.Wistia,
    )

    A_avg_node_sort = sorted(A_avg_node, reverse=True)
    A_avg_node_rank = [A_avg_node_sort.index(e) for e in A_avg_node]
    for x, y in zip([T + 5] * N, list(range(1, N + 1))):
        ax.text(
            x,
            y,
            str(A_avg_node_rank[y - 1] + 1),
            color='black',
            fontsize=10,
        )

    A_avg_time = cs['a_avg_time']
    ax.scatter(
        X,
        [N + 1] * (T + 1),
        s=A_avg_time * 100,
        c=A_avg_time,
        marker='s',
        alpha=0.5,
        cmap=cm.Wistia,
    )

    # Vertical date lines
    ax.text(0, -0.2, 'Mon', rotation=90)
    for i, point in enumerate(t_day_idx):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='green',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, t_day_week[i], rotation=90)
    for i, point in enumerate(t_hour_12):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='red',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, 'Noon', rotation=90, fontsize=6)
    for i, point in enumerate(t_hour_17):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='blue',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, '5 PM', rotation=90, fontsize=6)

    # X axes
    ax.set_xlabel('Time')
    # ax.set_xticks(range(0, T + 1, 10))
    # OR
    ax.set_xticks([1] + t_day_idx + [T + 1])
    ax.set_xticklabels([t_first] + t_day_date + [t_last])
    # ax_label_first = ax.get_xticklabels()[0]
    # ax_label_first.set_rotation(45)
    # ax_label_first.set_ha('left')
    # ax_label_last = ax.get_xticklabels()[-1]
    # ax_label_last.set_rotation(45)
    # ax_label_last.set_ha('left')

    # Y axes
    ax.set_ylabel('Node Index')
    ax.set_yticks(range(1, N + 1))

    # Figure labels
    fig_title = 'Authority Score Over Time'
    if len(label_output) > 0: fig_title = fig_title + ' (' + label_output + ')'
    ax.set_title(fig_title)

    # Color bar
    # fig.colorbar(mapper, ax=ax, shrink=0.5, pad=0.01)
    fig.colorbar(sc, ax=ax, shrink=0.5, pad=0.01)

    # Save figure
    fig.savefig(file_image[0], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)  # Close the figure window

    # HUB
    # ---

    H = cs['H']
    fig, ax = plt.subplots(figsize=(32, 20))
    for row in range(len(H)):
        ax.scatter(
            X,
            [row + 1] * (T + 1),
            s=H[row] * 1000,
            c=H[row],
            marker='|',
            alpha=0.8,
            norm=norm,
            cmap=mapper,
        )

    H_avg_node = cs['h_avg_node']
    ax.scatter(
        [T + 3] * N,
        list(range(1, N + 1)),
        s=H_avg_node * 100,
        c=H_avg_node,
        marker='s',
        alpha=0.5,
        cmap=cm.Wistia
    )

    H_avg_node_sort = sorted(H_avg_node, reverse=True)
    H_avg_node_rank = [H_avg_node_sort.index(e) for e in H_avg_node]
    for x, y in zip([T + 5] * N, list(range(1, N + 1))):
        ax.text(
            x,
            y,
            str(H_avg_node_rank[y - 1] + 1),
            color='black',
            fontsize=10,
        )

    H_avg_time = cs['h_avg_time']
    ax.scatter(
        X,
        [N + 1] * (T + 1),
        s=H_avg_time * 100,
        c=H_avg_time,
        marker='s',
        alpha=0.5,
        cmap=cm.Wistia,
    )

    ax.text(0, -0.2, 'Mon', rotation=90)
    for i, point in enumerate(t_day_idx):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='green',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, t_day_week[i], rotation=90)
    for i, point in enumerate(t_hour_12):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='red',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, 'Noon', rotation=90, fontsize=6)
    for i, point in enumerate(t_hour_17):
        ax.axvline(
            x=point,
            ymin=0.03,
            ymax=0.95,
            linewidth=0.5,
            color='blue',
            alpha=0.25
        )
        ax.text(point - 1, -0.2, '5 PM', rotation=90, fontsize=6)

    ax.set_xlabel('Time')
    ax.set_xticks([1] + t_day_idx + [T + 1])
    ax.set_xticklabels([t_first] + t_day_date + [t_last])
    ax.set_ylabel('Node Index')
    ax.set_yticks(range(1, N + 1))
    fig_title = 'Hub Score Over Time'
    if len(label_output) > 0: fig_title = fig_title + ' (' + label_output + ')'
    ax.set_title(fig_title)
    fig.colorbar(sc, ax=ax, shrink=0.5, pad=0.01)
    fig.savefig(file_image[1], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

    # REPORT
    # -------
    # Top node of temporal graph

    # Check the correctness value of section and top
    if top * section > N or top == -1:
        # Analyze all nodes
        top = N
        section = 1

    # Selected top nodes
    selected_a = []
    selected_h = []
    # Rank nodes based on average score
    idx_a, _ = list(zip(*rank(A_avg_node)))
    idx_h, _ = list(zip(*rank(H_avg_node)))

    # Select top 'n' nodes from each split
    for sp in np.array_split(idx_a, section):
        selected_a.extend(sp[:top])
    for sp in np.array_split(idx_h, section):
        selected_h.extend(sp[:top])

    df_columns = []
    df_labels = ['_a', '_h', '_id', '_od', '_iw', '_ow']
    df_index = []

    for node in selected_a:
        df_index.extend([str(node) + e for e in df_labels])
        col_in_d = []
        col_out_d = []
        col_in_w = []
        col_out_w = []
        for i in range(0, N * (T + 1), N):  # T + 1 iteration
            n = node + i  # Give node index over different times
            if graph.has_node(n):
                col_in_d.append(graph.in_degree(n))
                col_out_d.append(graph.out_degree(n))
                sum_w = 0
                for nbr in graph.predecessors(n):
                    sum_w += ew.get((nbr, n), 1)
                col_in_w.append(sum_w)
                sum_w = 0
                for nbr in graph.successors(n):
                    sum_w += ew.get((n, nbr), 1)
                col_out_w.append(sum_w)
            else:  # The node has been removed from graph
                col_in_d.append(0)
                col_out_d.append(0)
                col_in_w.append(0)
                col_out_w.append(0)
        df_columns.append(A[node])
        df_columns.append(H[node])
        df_columns.append(col_in_d)
        df_columns.append(col_out_d)
        df_columns.append(col_in_w)
        df_columns.append(col_out_w)

    for node in selected_h:
        df_index.extend([str(node) + e for e in df_labels])
        col_in_d = []
        col_out_d = []
        col_in_w = []
        col_out_w = []
        for i in range(0, N * (T + 1), N):
            n = node + i
            if graph.has_node(n):
                col_in_d.append(graph.in_degree(n))
                col_out_d.append(graph.out_degree(n))
                sum_w = 0
                for nbr in graph.predecessors(n):
                    sum_w += ew.get((nbr, n), 1)
                col_in_w.append(sum_w)
                sum_w = 0
                for nbr in graph.successors(n):
                    sum_w += ew.get((n, nbr), 1)
                col_out_w.append(sum_w)
            else:
                col_in_d.append(0)
                col_out_d.append(0)
                col_in_w.append(0)
                col_out_w.append(0)
        df_columns.append(A[node])
        df_columns.append(H[node])
        df_columns.append(col_in_d)
        df_columns.append(col_out_d)
        df_columns.append(col_in_w)
        df_columns.append(col_out_w)

    # Create and save dataframe of top nodes from each percentile (or split)
    df = pd.DataFrame(df_columns, index=df_index, columns=list(range(T + 1))).T
    df.to_csv(file_output[0], index=False)

    # Plot the report
    # df = df.t
    df = pd.read_csv(file_output[0]).T
    times = list(df.columns)
    idx = df.index.values.tolist()
    a_nodes, h_nodes = np.array_split(
        [int(idx[i].split('_')[0]) for i in range(0, len(idx), 6)], 2
    )

    # Authority scores
    fig, axs = plt.subplots(
        len(a_nodes),
        1,
        figsize=(16, 12),
        constrained_layout=True,
    )
    # cmap = plt.get_cmap('tab10')
    for i in range(len(a_nodes)):
        row_label = str(a_nodes[i]) + '_a'
        row_label2 = str(a_nodes[i]) + '_id'  # id,od,iw,ow
        legend_label = str(a_nodes[i])
        # axs[i].scatter(
        axs[i].plot(
            times,
            df.loc[row_label],
            # (0)
            c='black',
            alpha=0.5,
            # (1)
            # c=[np.random.rand(3, )],
            # (2)
            # c=[cmap(i)],
            # (1)
            # linewidth=1,
            # marker='o',
            # markersize=6,
            # (2)
            # marker='.',
            # label=legend_label
        )
        sc = axs[i].scatter(
            times,
            [0] * len(times),
            # df.loc[row_label2],
            s=df.loc[row_label2] * 100,
            c=df.loc[row_label2],
            marker='|',
            alpha=1.0,
            vmin=0,
            # vmin=min(df.loc[row_label2]),
            vmax=28,
            # vmax=max(df.loc[row_label2]),
            cmap=cm.Wistia,
        )
        axs[i].set_ylabel('Node ' + str(a_nodes[i]))
    # fig.legend(loc='center left', bbox_to_anchor=(1.03, 0.5))
    fig.text(0.5, 1.02, 'Temporal HITS', ha='center')
    fig.text(0.5, -0.02, 'Time', ha='center')
    fig.text(-0.02, 0.5, 'Authority Score', va='center', rotation='vertical')
    fig.colorbar(sc, ax=ax, shrink=0.5, pad=0.01)
    fig.savefig(file_image[2], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

    # Hub scores
    fig, axs = plt.subplots(
        len(h_nodes),
        1,
        figsize=(16, 12),
        constrained_layout=True,
    )
    for i in range(len(h_nodes)):
        row_label = str(h_nodes[i]) + '_h'
        row_label2 = str(h_nodes[i]) + '_od'  # id,od,iw,ow
        axs[i].plot(
            times,
            df.loc[row_label],
            c='black',
            alpha=0.5,
        )
        sc = axs[i].scatter(
            times,
            [0] * len(times),
            s=df.loc[row_label2] * 100,
            c=df.loc[row_label2],
            marker='|',
            alpha=1.0,
            vmin=0,
            vmax=28,
            cmap=cm.Wistia,
        )
        axs[i].set_ylabel('Node ' + str(a_nodes[i]))
    fig.text(0.5, 1.02, 'Temporal HITS', ha='center')
    fig.text(0.5, -0.02, 'Time', ha='center')
    fig.text(-0.02, 0.5, 'Hub Score', va='center', rotation='vertical')
    fig.colorbar(sc, ax=ax, shrink=0.5, pad=0.01)
    fig.savefig(file_image[3], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

    # TOP
    # ---
    # Top nodes of TON graph

    if report_num > N * (T + 1) or report_num == -1:
        # Analyze all nodes
        report_num = N * (T + 1)

    # Top rank TON graph nodes based on HITS score
    A_top = rank(a, return_rank=True)[:report_num]
    H_top = rank(h, return_rank=True)[:report_num]

    df_index = []
    col_r = []  # Rank
    col_c = []  # Category i.e. A or H
    col_n = []  # Node id
    col_a = []
    col_h = []
    col_p = []  # Parent
    col_t = []  # Time
    col_in_d = []
    col_out_d = []
    col_in_w = []
    col_out_w = []

    for n, r in A_top.items():
        df_index.append('a_' + str(n))
        col_n.append(n)
        col_r.append(r)
        col_c.append('a')
        col_a.append(a[n])
        col_h.append(h[n])
        col_p.append(n % N)
        col_t.append(n // N)
        col_in_d.append(graph.in_degree(n))
        col_out_d.append(graph.out_degree(n))
        sum_w = 0
        for nbr in graph.predecessors(n):
            sum_w += ew.get((nbr, n), 1)
        col_in_w.append(sum_w)
        sum_w = 0
        for nbr in graph.successors(n):
            sum_w += ew.get((n, nbr), 1)
        col_out_w.append(sum_w)

    for n, r in H_top.items():
        df_index.append('h_' + str(n))
        col_n.append(n)
        col_r.append(r)
        col_c.append('h')
        col_a.append(a[n])
        col_h.append(h[n])
        col_p.append(n % N)
        col_t.append(n // N)
        col_in_d.append(graph.in_degree(n))
        col_out_d.append(graph.out_degree(n))
        sum_w = 0
        for nbr in graph.predecessors(n):
            sum_w += ew.get((nbr, n), 1)
        col_in_w.append(sum_w)
        sum_w = 0
        for nbr in graph.successors(n):
            sum_w += ew.get((n, nbr), 1)
        col_out_w.append(sum_w)

    df = pd.DataFrame(
        {
            'r': col_r,
            'n': col_n,
            'a': col_a,
            'h': col_h,
            'p': col_p,
            't': col_t,
            'id': col_in_d,
            'od': col_out_d,
            'iw': col_in_w,
            'ow': col_out_w,
            'c': col_c
        },
        index=df_index
    )

    # Save top node analysis
    df.to_csv(file_output[1], index=False)

    # Correlation between in/out-degree and HITS scores
    fig, axs = plt.subplots(2, 2, figsize=(16, 16), constrained_layout=True)
    axs[0, 0].scatter(df[df.c == 'a'].id, df[df.c == 'a'].a)
    axs[0, 0].set_xlabel('In-degree')
    axs[0, 0].set_ylabel('Authority Score')
    axs[0, 1].scatter(df[df.c == 'a'].od, df[df.c == 'a'].a)
    axs[0, 1].set_xlabel('Out-degree')
    axs[0, 1].set_ylabel('Authority Score')
    axs[1, 0].scatter(df[df.c == 'a'].iw, df[df.c == 'a'].a)
    axs[1, 0].set_xlabel('In-weight')
    axs[1, 0].set_ylabel('Authority Score')
    axs[1, 1].scatter(df[df.c == 'a'].ow, df[df.c == 'a'].a)
    axs[1, 1].set_xlabel('Out-weight')
    axs[1, 1].set_ylabel('Authority Score')
    fig.savefig(file_image[4], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

    fig, axs = plt.subplots(2, 2, figsize=(16, 16), constrained_layout=True)
    axs[0, 0].scatter(df[df.c == 'h'].id, df[df.c == 'h'].h)
    axs[0, 0].set_xlabel('In-degree')
    axs[0, 0].set_ylabel('Hub Score')
    axs[0, 1].scatter(df[df.c == 'h'].od, df[df.c == 'h'].h)
    axs[0, 1].set_xlabel('Out-degree')
    axs[0, 1].set_ylabel('Hub Score')
    axs[1, 0].scatter(df[df.c == 'h'].iw, df[df.c == 'h'].h)
    axs[1, 0].set_xlabel('In-weight')
    axs[1, 0].set_ylabel('Hub Score')
    axs[1, 1].scatter(df[df.c == 'h'].ow, df[df.c == 'h'].h)
    axs[1, 1].set_xlabel('Out-weight')
    axs[1, 1].set_ylabel('Hub Score')
    fig.savefig(file_image[5], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

    # Correlation matrix
    corr = df.corr()  # method='pearson',
    # corr = df.corr(method ='kendall')
    # corr = df.corr(method ='spearman')
    fig, ax = plt.subplots(figsize=(9, 8))
    sns.heatmap(corr, ax=ax, cmap='YlGnBu', linewidths=0.1)
    fig.savefig(file_image[6], dpi=300, bbox_inches='tight', transparent=True)
    plt.close(fig)

## Test

In [183]:
# a, h = hits(
#     file_input=[
#         'network/bt_tn_full_network.gpickle',
#         'network/bt_tn_light_network.gpickle',
#         'network/bt_temporal_times.csv',
#         'network/bt_temporal_nodes.csv',
#         'network/bt_tn_weights.csv',
#     ],
#     file_output=[
#         'hits/a.csv',
#         'hits/h.csv',
#     ],
#     label_input='',
#     label_output='',
#     graph=None,
#     times=None,
#     nodes=None,
#     ew=None,
#     nstart=None,
#     full=False,
#     version=3,  # 0
#     sigma=0.85,
#     max_iter=100,
#     tol=1.0e-8,
#     norm_max=True,
#     norm_final_l1=True,
#     norm_final_l2=False,
#     norm_iter=False,
#     norm_degree=False,
#     norm_damping=False,
#     output=True,
#     plot=True,
#     save=False
# )

In [ ]:
# a, h = hits_read()

In [221]:
# hits_conditional()

In [254]:
# hits_analyze(
#     file_input=[
#         'network/bt_temporal_nodes.csv',
#         'network/bt_temporal_times.csv',
#     ],
#     file_net=['network/bt_tn_light_network.gpickle'],
#     file_ew=['network/bt_tn_weights.csv'],
#     file_hits=[
#         'hits/a.csv',
#         'hits/h.csv',
#         'hits/a_array.csv',
#         'hits/h_array.csv',
#         'hits/a_avg_node.csv',
#         'hits/a_avg_time.csv',
#         'hits/h_avg_node.csv',
#         'hits/h_avg_time.csv',
#         'hits/a_norm_node.csv',
#         'hits/a_norm_time.csv',
#         'hits/h_norm_node.csv',
#         'hits/h_norm_time.csv',
#     ],
#     file_output=['hits/report.csv', 'hits/top.csv'],
#     file_image=[
#         'hits/fig_a.pdf',
#         'hits/fig_h.pdf',
#         'hits/fig_a_report.pdf',
#         'hits/fig_h_report.pdf',
#         'hits/fig_a_corr.pdf',
#         'hits/fig_h_corr.pdf',
#         'hits/fig_mat.pdf',
#     ],
#     label_input='',
#     label_hits='',
#     label_output='',
#     label_image='',
#     graph=None,
#     times=None,
#     nodes=None,
#     ew=None,
#     a=None,
#     h=None,
#     top=2,
#     section=4,
#     report_num=100
# )

## Group Test
Calculate a series of different HITS versions (1 ... 9), then compute conditional scores and analyze the top rank nodes and finally plot the result

In [ ]:
l = 3
ll = 'v' + str(l)
# A, H = hits(graph=None,times=None,nodes=None,edge_weight=None,nstart=None,label_input='',label_output=ll,_full=False,_max_iter=300,_tol=1.0e-8,_version=l,_sigma=0.85,_norm_max=True,_norm_final_l1=True,_norm_final_l2=False,_norm_iter=False,_norm_degree=False,_norm_damping=False,_norm_round=False,_round_num=5,save=True)
hits_top(label_input='',label_output=ll,label_hits=ll,label_net='',label_ew='',top=10,section=1,report_num=100,round_num=5)

In [ ]:
# for l in range(1,10):
#     ll = 'v' + str(l)
#     A, H = hits(graph=None,times=None,nodes=None,edge_weight=None,nstart=None,label_input='',label_output=ll,_full=False,_max_iter=300,_tol=1.0e-8,_version=l,_sigma=0.85,_norm_max=True,_norm_final_l1=True,_norm_final_l2=False,_norm_iter=False,_norm_degree=False,_norm_damping=False,_norm_round=False,_round_num=5,save=True)
#     hits_top(label_input='',label_output=ll,label_hits=ll,label_net='',label_ew='',top=10,section=1,report_num=100,round_num=5)
#     print('\n===\n')

# Node Removal

In [276]:
def tn_bt_to_temporal(
    file_input=[
        'remove/bt_tn_network.gpickle', 'network/bt_temporal_nodes.csv',
        'network/bt_temporal_times.csv'
    ],
    file_output=[
        'remove/bt_temporal_network.gpickle', 'remove/bt_temporal_times.csv',
        'remove/bt_temporal_nodes.csv'
    ],
    label_input='',
    label_output='',
    label_graph_name='',
    temporal=None,
    trans_remove=True,
    output_network=True,
    save_times=True,
    save_nodes=True,
    save_network_file=True
):
    """
    Convert a (directed) time-ordered network (TON) to a (multi-edge) temporal (T) network
    """
    # Amend filenames
    file_input = label_amend(file_input, label_input)
    file_output = label_amend(file_output, label_output)

    # Create empty TEMPORAL network
    graph = nx.MultiDiGraph()
    graph.name = 'Temporal Network (' + label_graph_name + ')'

    # Read temporal networks from file
    if temporal is None:
        temporal = nx.read_gpickle(file_input[0])

    # Nodes
    N = line_count(file_input[1])

    # Timestamp
    times = temporal_bt_times_read()  # TODO: we may not need it
    T = line_count(file_input[2])

    times_set = set()
    for u, v, data in temporal.edges(data=True):
        parent_u = u % N
        parent_v = v % N
        time_uv = u // N  # OR v // N - 1
        time_delta = abs(v - u) // N
        # Crossed edge
        if parent_u != parent_v:  # and time_delta == 1:
            # print(f'{parent_u} -> {parent_v} with {data} at time {times.loc[time_uv]}')
            if trans_remove and data.get('trans', False):
                # If the the edge is transitive and we want to ignore trans -> skip
                continue
            graph.add_edge(
                parent_u, parent_v, t=times.loc[time_uv], w=data['w']
            )
            # Save timestamp to the new time set
            times_set.add(times.loc[time_uv])

    # Convert times set to series and save
    times_new = pd.Series(sorted(list(times_set)))
    nodes_new = pd.Series(sorted(list(graph.nodes)))

    # Save graph
    if save_network_file: nx.write_gpickle(graph, file_output[0])

    # Save times
    if save_times:
        np.savetxt(file_output[1], times_new, delimiter=',', fmt='%s')

    # Save nodes
    if save_nodes:
        np.savetxt(file_output[2], nodes_new, delimiter=',', fmt='%s')

    # Print network statistics
    if output_network:
        print(nx.info(graph))
        print(f'Number of times: {len(times_new)}')

    return graph

In [279]:
def hits_remove(
    a=None,
    h=None,
    graph=None,
    times=None,
    nodes=None,
    ew=None,
    file_input=[
        'network/bt_temporal_times.csv',
        'network/bt_temporal_nodes.csv',
    ],
    file_hits=[
        'hits/a.csv',
        'hits/h.csv',
    ],
    file_output=[
        'remove/bt_tn_network.gpickle',
        'remove/bt_temporal_network.gpickle',
        'remove/bt_temporal_times.csv',
        'remove/bt_temporal_nodes.csv',
        'remove/bt_tn_weights.csv',
        'remove/bt_tn_probs.csv',
        'remove/a.csv',
        'remove/h.csv',
        'remove/a_array.csv',
        'remove/h_array.csv',
        'remove/a_avg_node.csv',
        'remove/a_avg_time.csv',
        'remove/h_avg_node.csv',
        'remove/h_avg_time.csv',
        'remove/a_norm_node.csv',
        'remove/a_norm_time.csv',
        'remove/h_norm_node.csv',
        'remove/h_norm_time.csv',
        'remove/report.csv',
        'remove/top.csv',
        'remove/fig_a.pdf',
        'remove/fig_h.pdf',
        'remove/fig_a_report.pdf',
        'remove/fig_h_report.pdf',
        'remove/fig_a_corr.pdf',
        'remove/fig_h_corr.pdf',
        'remove/fig_mat.pdf',
        'remove/remove.csv',
    ],
    label_input='',
    label_hits='',
    label_net='',
    label_ew='',
    label_output='',
    label_output_folder='',
    epoch=10,  # How many times repead the node removal action
    remove=0.5,  # Stop if X ratio of nodes were removed (even if not reached X epoch)
    step=10,  # E.g. Remove 10 % or 10 nodes at each epoch    
    strategy_a='a',  # 'a' or 'h' = authority or hub    
    strategy_b='t',  # 't' or 's' = temporal or static
    strategy_c='r',  # 'r' or 'n' = ratio or number
    strategy_d=1,  # Score-based or random approach
    time_window=(5, 12),  # Remove high rank nodes between 5 pm - 12 am
    rd=8,  # Decimal point
    actions=[0, 2],
    output=True,
    plot_times=False,
    save_networks=True,
    return_graphs=True,
    return_scores=True,
):
    """
    Strategy
        A: Score
            a = authority
            h = hub
        B: Network
            s = static
            t = temporal
        C: Size
            n = number number of nodes to be removed (default = 1)
            r = ratio of nodes to be removed (default = 1 %)
        D: method
            0) randomly
            1) centrality-based (i.e. temporal HITS)
            2) high rank nodes at specific time window
            3) degree-based (TODO in future)
    Actions
        0) Convert TON to TEMPORAL model and save it
        1) Convert TEMPORAL model to STATIC model
        2) Calculate temporal HITS on modified network
            A) Calculate new Edge-Weghts and save
            B) Calculate HITS
            C) Analyze HITS
        ---
        TODO
        2) intersection similarity
        3) centrality robustness
        4) influence maximization
        5) network diameter (90 threshhold)
        6) Number and size of gient connected components (CC)
            - Is it still one big CC (time = 1 -> T) or it is broken into pices
            - How many CC's or time windows (e.g. start-t2, t2-t10, t10-end)
        7) Average pair-wise (tempora and topological=hop) distance
        8) Network reachability
        9) Epidemic treshhold (spread of information or disease)
        10) Shanon diversity
    """

    # Edit files
    file_input = label_amend(file_input, label_input)
    file_hits = label_amend(file_hits, label_hits)
    file_output = label_amend(file_output, label_output)

    # Read times
    if times is None:
        times = list(temporal_bt_times_read([file_input[0]]))
        T = line_count(file_input[0])
    else:
        T = len(times)

    # Read nodes
    if nodes is None:
        nodes = list(temporal_bt_nodes_read([file_input[1]]))
        N = line_count(file_input[1])
    else:
        N = len(nodes)
        # N = graph.number_of_nodes() // T

    # Read network
    if graph == None:
        graph = tn_bt_light_read(label_input=label_net)

    # Number of nodes and edges of TON graph
    N_new = graph.number_of_nodes()
    M_new = graph.number_of_edges()

    # Size of nodes and timestamp of TON after node removal
    # Will be updated later (better to have it as global variable)
    M = 0
    temporal_N = 0
    temporal_T = 0

    # Read edge weights
    if ew is None: ew = ew_read(label_input=label_ew)  # TODO: can be removed

    # Print network info before any node removal
    if output: print(nx.info(graph), '\n')

    # Read HITS scores
    if a is None or h is None:
        a, h = hits_read(
            file_input=file_hits,
            label_input='',
            output=False,
            plot=False,
        )


#     # Set rounding number (decimal point) for HITS scores
#     fmt_str = '%1.' + str(rd) + 'f'

# Create list of scores by sorting the key of dictionary
    a_values, h_values = [], []
    for k in sorted(a.keys(), reverse=False):  # Ascending
        # Score of node 1 to the last node id number
        a_values.append(a[k])
        h_values.append(h[k])

    # Create A and H matrices by iterating through sorted list of node
    A = np.reshape(a_values, (T + 1, N)).T
    H = np.reshape(h_values, (T + 1, N)).T

    # Create average score of nodes and timestamps
    A_avg_node = A.sum(axis=1) / (T + 1)
    A_avg_time = A.sum(axis=0) / N
    H_avg_node = H.sum(axis=1) / (T + 1)
    H_avg_time = H.sum(axis=0) / N

    # Time Analysis
    t_first = times[0].strftime('%d %B\n%I %p')
    t_last = times[-1].strftime('%d %B\n%I %p')
    t_0 = times[0] - pd.Timedelta(1, unit='h')

    # Distribution of timestamps over days and weeks of the month
    # Also finding time index of timestamp that hour change to 7am, 12p, 5pm, and 12am
    t_day_idx = []
    t_day_week = []
    t_day_date = []  # 7 am (or earliest time of the day)
    t_hour_12 = []
    t_hour_17 = []
    # Save index of each timestamp belong to what time windows of the day
    # [ [7 am - noon], [12 pm - 5 pm], [5 pm - 12 am] ]
    for i, time in enumerate(times):
        if t_0.weekday() != time.weekday():  # When day of the week changes
            t_day_idx.append(
                i
            )  # Index of the first hour of each day (expected to be 7 am)
            t_day_week.append(time.strftime('%a'))  # Monday ...
            t_day_date.append(time.strftime('%d'))  # 1st, 2nd, ..., 31
            t_0 = time
        if time.strftime('%H') == '12':
            t_hour_12.append(i)
        if time.strftime('%H') == '17':
            t_hour_17.append(i)

    # Adding first timestamp same as timestamp 1
    # Just because of temporal network model requires this method
    times.insert(0, times[0])  # Make times list to size T + 1

    # Best hour and day in terms of highest authority average score
    days_a = defaultdict(list)
    hours_a = defaultdict(list)
    for i, score in enumerate(A_avg_time):
        days_a[times[i].weekday()].append(score)
        hours_a[times[i].hour].append(score)
    days_a_sum = {k: sum(v) / len(v) for k, v in days_a.items()}
    hours_a_sum = {k: sum(v) / len(v) for k, v in hours_a.items()}

    # Day-Hour matrix score
    # 7 days and 18 hours [7 am ... 11 pm] , missing 6 hours of [12 am ... 6 am]
    day_hour_a = np.zeros((7, 18))
    day_hour_a_count = np.zeros((7, 18))
    for i, t in enumerate(times):
        day_hour_a[t.dayofweek][t.hour - 6] += A_avg_time[i]
        day_hour_a_count[t.dayofweek][t.hour - 6] += 1
    # To avoid ZeroDivisionError, replace all 0 with 1
    # Still average produce 0 because observe/count (0/1=0)
    day_hour_a_count_new = day_hour_a_count.copy()
    day_hour_a_count_new[day_hour_a_count == 0] = 1
    day_hour_a_avg = np.divide(day_hour_a, day_hour_a_count_new)[:, 1:]

    # Best hour and day in terms of highest hub average score
    days_h = defaultdict(list)
    hours_h = defaultdict(list)
    for i, score in enumerate(H_avg_time):
        days_h[times[i].weekday()].append(score)
        hours_h[times[i].hour].append(score)
    days_h_sum = {k: sum(v) / len(v) for k, v in days_h.items()}
    hours_h_sum = {k: sum(v) / len(v) for k, v in hours_h.items()}

    # Day-Hour matrix score
    day_hour_h = np.zeros((7, 18))
    day_hour_h_count = np.zeros((7, 18))
    for i, t in enumerate(times):
        day_hour_h[t.dayofweek][t.hour - 6] += H_avg_time[i]
        day_hour_h_count[t.dayofweek][t.hour - 6] += 1

    day_hour_h_count_new = day_hour_h_count.copy()
    day_hour_h_count_new[day_hour_h_count == 0] = 1
    day_hour_h_avg = np.divide(day_hour_h, day_hour_h_count_new)[:, 1:]

    if plot_times:
        ax = plt.axes()
        sns.heatmap(
            day_hour_a_count[:, 1:],
            linewidth=0.5,
            cmap='YlGnBu',
            xticklabels=list(range(7, 24)),
            yticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            ax=ax
        )
        ax.set_title('Day and hour frequency of timestamps')
        plt.show()
        # ---
        ax = plt.axes()
        sns.heatmap(
            day_hour_a_avg,
            linewidth=0.5,
            cmap='YlGnBu',
            xticklabels=list(range(7, 24)),
            yticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        )
        ax.set_title('Average authority score over days and hours')
        plt.show()
        # ---
        ax = plt.axes()
        sns.heatmap(
            day_hour_h_avg,
            linewidth=0.5,
            cmap='YlGnBu',
            xticklabels=list(range(7, 24)),
            yticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        )
        ax.set_title('Average hub score over days and hours')
        plt.show()

    epoch_0 = 0  # How many times the algorimth is looped
    remove_0 = 0  # ratio of removed nodes

    # Number of nodes or ratio
    if strategy_c == 'n':
        # Step should be an integer number
        # print(f'Removing {step} nodes or equevelant of {step/N_new*100:.2f} % of all the nodes at every epoch\n')
        pass  # Do nothing, if not printing
    elif strategy_c == 'r':
        # Step should be a float ratio [0.0  ... 1.0]
        # print(f'Removing of {step*100:.2f} % of all the nodes or equevelant of {int(N_new*step)} nodes at every epoch\n')
        # Edit number of removing nodes based on input ratio
        step = int(N_new * step)

    # Iterative over chunk of X nodes with high HITS scores
    bd = []
    tops = []
    if strategy_a == 'a':
        bd = breakdown(a_values, step)
        tops = iter(bd)
    elif strategy_a == 'h':
        bd = breakdown(h_values, step)
        tops = iter(bd)

    # If wants to return the calculated graphs at the end
    graphs = {}  # Dict of temporal graphs
    tons = {}  # Dict of TON model graphs
    auts = {}  # Dict of authority scores
    hubs = {}  # Dict of hub scores

    # Add initial graphs and scores here
    temporal = temporal_bt_read()
    M = temporal.number_of_edges()

    if return_graphs:
        tons[epoch_0] = graph
        graphs[epoch_0] = temporal
    if return_scores:
        auts[epoch_0] = a
        hubs[epoch_0] = h

    epoch_0 += 1  # 1
    selected_nodes = []
    removed_total = []
    # Keep removing nodes until one of the conditions happen
    # while epoch_0 <= epoch and remove_0 <= remove:
    for _ in range(1, len(bd)):
        if epoch_0 > epoch or remove_0 >= remove:
            break
        else:
            if output: print(f'EPOCH {epoch_0}:\n---------')

        # Create saving folders and file names
        file_output_new = []
        if not len(label_output_folder) > 0:  # Empty
            label_output_folder_temp = strategy_a + '-' + strategy_c + '-' + str(
                strategy_d
            ) + '-' + str(epoch_0)
            file_output_new = label_amend(
                file_output, label_output_folder_temp, end=False
            )
        else:
            file_output_new = file_output

        # Empty list of node to be removed ...
        selected_nodes.clear()

        # Method 0
        if strategy_d == 0:
            if output:
                print(
                    'Removing', step, 'nodes out of', graph.number_of_nodes(),
                    'selected randomly ...\n'
                )

            # Pick a set of nodes uniformy random
            # np.random.seed(0)
            selected_nodes = list(
                np.random.choice(graph.nodes, size=step, replace=False)
            )
            removed_total.extend(selected_nodes)
            np.savetxt(
                file_output_new[27],
                selected_nodes,
                delimiter=',',
                fmt='%s',
            )
            # if output: print(selected_nodes)

        # Method 1
        elif strategy_d == 1:
            if output:
                print(
                    'Removing', step, 'nodes out of', graph.number_of_nodes(),
                    'selected based on temporal HITS scores ...\n'
                )

            # Select top ranked HITS score nodes
            selected_nodes = next(tops)
            removed_total.extend(selected_nodes)
            np.savetxt(
                file_output_new[27],
                selected_nodes,
                delimiter=',',
                fmt='%s',
            )
            # if output: print(selected_nodes)

        # Method 3 TODO
        elif strategy_d == 2:
            if output:
                print(
                    'Removing', step, 'nodes out of', graph.number_of_nodes(),
                    'selected based on average HITS scores and in time window of ',
                    time_window, ' ...\n'
                )

            # Sort best node with highest average score
            ids_a = A_avg_node.argsort()[::-1]
            ids_h = H_avg_node.argsort()[::-1]
            # Select based on desired ratio
            selected_a = ids_a[epoch * step:(epoch + 1) * step]
            selected_h = ids_h[epoch * step:(epoch + 1) * step]
            # Filter for desired time window
            selected_noodes = []
            selected_times = list(range(T + 1))
            for n in selected_a:
                selected_noodes.extend([N * t + n for t in selected_times])
            removed_total.extend(selected_nodes)

        # Remove selected nodes
        graph.remove_nodes_from(selected_nodes)
        graph.name = 'Time-ordered Network (' + str(epoch_0) + ')'

        # If returning
        if return_graphs: tons[epoch_0] = graph

        # Save
        if save_networks: nx.write_gpickle(graph, file_output_new[0])

        # Print network statistics after node removal
        if output:
            print(nx.info(graph))
            print(
                f'Removed {N_new - graph.number_of_nodes()} nodes out of {N_new}'
                f' or {(N_new - graph.number_of_nodes()) / N_new * 100:.2f} % and {M_new - graph.number_of_edges()}'
                f' edges out of {M_new} or {(M_new - graph.number_of_edges()) / M_new * 100:.2f} %'
            )
            print()

        # ACTIONS
        # -------

        # Convert TON to TEMPORAL then save
        if 0 in actions:
            temporal = tn_bt_to_temporal(
                temporal=graph,
                file_output=file_output_new[1:4],  # 1...3
                label_graph_name=str(epoch_0)
            )
            # update number of nodes and timestamp in temporal graph (not TON)
            temporal_T = line_count(file_output_new[2])
            temporal_N = line_count(file_output_new[3])
            if output:
                # print(nx.info(temporal))
                print(
                    f'Removed {N - temporal_N} nodes out of {N}'
                    f' or {(N - temporal_N) / N*100:.2f} % and {M - temporal.number_of_edges()}'
                    f' edges out of {M} or {(M - temporal.number_of_edges()) / M * 100:.2f} %'
                )
                print()
            # If returning
            if return_graphs: graphs[epoch_0] = temporal

        # Convert TEMPORAL to STATIC then save
        if 1 in actions:
            pass

        # Calculate HITS on new TON graph
        if 2 in actions:
            # Edge-Weight
            ew = ew_create(
                file_input=[
                    file_output_new[0],  # graph
                    'network/bt_temporal_nodes.csv',  # original nodes
                    'network/bt_temporal_times.csv',  # original times
                ],
                file_output=[file_output_new[4]],  # edge-weights
                # graph=graph, # after removal
                # number_of_nodes=N,  # original
                # number_of_times=T,  # original
                version=3,  # 0
                omega=1,
                gamma=0.0001,
                epsilon=1,
                distance=0.1,  # 1
                alpha=0.5,
                save_weights=True,
                output_weights=False,
                plot_weights=False
            )
            # HITS
            a_n, h_n = hits(
                file_input=[
                    'network/bt_tn_full_network.gpickle',  # not needed
                    file_output_new[0],  # graph after removal
                    'network/bt_temporal_times.csv',  # original nodes
                    'network/bt_temporal_nodes.csv',  # original times
                    file_output_new[4],  # edge weights after removal
                ],
                file_output=file_output_new[6:8],  # 6...7
                label_output='',
                # graph=graph,  # after removal
                # times=times,  # original times list
                # nodes=nodes,  # original nodes list
                # ew=ew,  # after removal
                version=3,  # 0
                sigma=0.85,
                max_iter=100,
                tol=1.0e-8,
                norm_max=True,
                norm_final_l1=True,
                norm_final_l2=False,
                norm_iter=False,
                norm_degree=False,
                norm_damping=False,
                output=False,
                plot=False,
                save=True
            )
            auts[epoch_0] = a_n
            hubs[epoch_0] = h_n
            # Conditional HITS scores
            hits_conditional(
                file_input=file_output_new[6:8],
                file_output=file_output_new[8:18],  # 8..17
                label_input='',
                label_output='',
                # a=a_n,
                # h=h_n,
                # N=N,
                # T=T,
                removed=True,
                save=True
            )
            # Analyze HITS
            hits_analyze(
                file_input=[
                    'network/bt_temporal_nodes.csv',
                    'network/bt_temporal_times.csv',
                ],
                file_net=[file_output_new[0]],  # graph after removal
                file_ew=[file_output_new[4]],  # edge weigths after removal
                file_hits=file_output_new[6:18],  # 6...17
                file_output=file_output_new[18:20],  # 18...19
                file_image=file_output_new[20:27],  # 20...26
                label_input='',
                label_hits='',
                label_output='',
                label_image='',
                # graph=graph,  # graph updated
                # times=times,  # original times
                # nodes=nodes,  # original nodes
                # ew=ew,  # edge weights updated
                # a=a_n,  # new authority scores
                # h=h_n,  # new hub scores
                top=2,
                section=4,
                report_num=100
            )

        # Update epoch and ...
        epoch_0 += 1
        remove_0 = (N_new - graph.number_of_nodes()) / N_new

    return graphs

In [285]:
g = hits_remove(
    epoch=30,  # Repeat for 1 epoch
    remove=0.5,  # Stop when reach 10 % removed
    step=100,  # Remove 100 nodes at every epoch
    strategy_a='a',  # Use authority
    strategy_b='t',  # Use temporal
    strategy_c='n',  # Use ratio
    strategy_d=1,  # Score-based
    actions=[0, 2],
    save_networks=True,
    output=True
)

Name: Time-ordered Network
Type: DiGraph
Number of nodes: 8428
Number of edges: 21384
Average in degree:   2.5373
Average out degree:   2.5373 

EPOCH 1:
---------
Removing 100 nodes out of 8428 selected based on temporal HITS scores ...

Name: Time-ordered Network (1)
Type: DiGraph
Number of nodes: 8328
Number of edges: 19827
Average in degree:   2.3808
Average out degree:   2.3808
Removed 100 nodes out of 8428 or 1.19 % and 1557 edges out of 21384 or 7.28 %

Name: Temporal Network (1)
Type: MultiDiGraph
Number of nodes: 28
Number of edges: 9250
Average in degree: 330.3571
Average out degree: 330.3571
Number of times: 297
Removed 0 nodes out of 28 or 0.00 % and 1144 edges out of 10394 or 11.01 %

EPOCH 2:
---------
Removing 100 nodes out of 8328 selected based on temporal HITS scores ...

Name: Time-ordered Network (2)
Type: DiGraph
Number of nodes: 8228
Number of edges: 19017
Average in degree:   2.3113
Average out degree:   2.3113
Removed 200 nodes out of 8428 or 2.37 % and 2367 edg

EPOCH 16:
---------
Removing 100 nodes out of 6928 selected based on temporal HITS scores ...

Name: Time-ordered Network (16)
Type: DiGraph
Number of nodes: 6828
Number of edges: 9248
Average in degree:   1.3544
Average out degree:   1.3544
Removed 1600 nodes out of 8428 or 18.98 % and 12136 edges out of 21384 or 56.75 %

Name: Temporal Network (16)
Type: MultiDiGraph
Number of nodes: 28
Number of edges: 2329
Average in degree:  83.1786
Average out degree:  83.1786
Number of times: 138
Removed 0 nodes out of 28 or 0.00 % and 8065 edges out of 10394 or 77.59 %

EPOCH 17:
---------
Removing 100 nodes out of 6828 selected based on temporal HITS scores ...

Name: Time-ordered Network (17)
Type: DiGraph
Number of nodes: 6728
Number of edges: 8769
Average in degree:   1.3034
Average out degree:   1.3034
Removed 1700 nodes out of 8428 or 20.17 % and 12615 edges out of 21384 or 58.99 %

Name: Temporal Network (17)
Type: MultiDiGraph
Number of nodes: 28
Number of edges: 2046
Average in degree:

/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


EPOCH 25:
---------
Removing 100 nodes out of 6028 selected based on temporal HITS scores ...

Name: Time-ordered Network (25)
Type: DiGraph
Number of nodes: 5928
Number of edges: 5413
Average in degree:   0.9131
Average out degree:   0.9131
Removed 2500 nodes out of 8428 or 29.66 % and 15971 edges out of 21384 or 74.69 %

Name: Temporal Network (25)
Type: MultiDiGraph
Number of nodes: 0
Number of edges: 0

Number of times: 0
Removed 28 nodes out of 28 or 100.00 % and 10394 edges out of 10394 or 100.00 %



/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


...

/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


EPOCH 29:
---------
Removing 100 nodes out of 5628 selected based on temporal HITS scores ...

Name: Time-ordered Network (29)
Type: DiGraph
Number of nodes: 5528
Number of edges: 5063
Average in degree:   0.9159
Average out degree:   0.9159
Removed 2900 nodes out of 8428 or 34.41 % and 16321 edges out of 21384 or 76.32 %

Name: Temporal Network (29)
Type: MultiDiGraph
Number of nodes: 0
Number of edges: 0

Number of times: 0
Removed 28 nodes out of 28 or 100.00 % and 10394 edges out of 10394 or 100.00 %



/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


EPOCH 30:
---------
Removing 100 nodes out of 5528 selected based on temporal HITS scores ...

Name: Time-ordered Network (30)
Type: DiGraph
Number of nodes: 5428
Number of edges: 4977
Average in degree:   0.9169
Average out degree:   0.9169
Removed 3000 nodes out of 8428 or 35.60 % and 16407 edges out of 21384 or 76.73 %

Name: Temporal Network (30)
Type: MultiDiGraph
Number of nodes: 0
Number of edges: 0

Number of times: 0
Removed 28 nodes out of 28 or 100.00 % and 10394 edges out of 10394 or 100.00 %



/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/alif/venv/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [ ]:
# len(g)

# print(nx.info(g[45]))

# list(g[45].edges(data=True))
list(g[47].edges(data=True)) # 47 is the network where its corresponding TON graph is broken in a way that there is no crossed edges left

In [ ]:
# hits_remove(
#     label_input='',
#     label_hits='',
#     label_net='',
#     label_ew='',
#     label_output='',
#     label_folder='',
#     # ---
#     epoch=1000,  # Repeat for 1000 epoch
#     remove=0.1,  # Stop when reach 10 % removed
#     step=0.01,  # Remove 1 % every epoch
#     strategy_a='a',  # Use authority
#     strategy_b='t',  # Use temporal
#     strategy_c='r',  # Use ratio
#     strategy_d=1,  # Score-based
#     time_window=(5, 12),  # Remove high rank nodes between 5 pm - 12 am
#     actions=[1],
#     # ---
#     save_networks=True,
#     save_hits=True,
#     # ---
#     output_changes=True)

# Intersection Similarity

$isim_K(L^1,L^2)=\frac{1}{k}\sum_{i=1}^{k}\frac{|L_{i}^{1}\Delta L_{i}^{2}|}{2i}$

In [ ]:
def isim(list1, list2):
    '''
    Calculate intercention similarity of two input lists
    '''
    # check if both list have the same size
    if len(list1) < len(list2):
        list2 = list2[:len(list1)]
    else:
        list1 = list1[:len(list2)]

    isim = []
    for i in range(1, len(list1) + 1):
        set1 = set(list1[:i])
        set2 = set(list2[:i])
        set_dif = set1 ^ set2  # symmetric difference
        isim.append(len(set_dif) / (2 * i))

    isim_norm = []
    for i in range(len(isim)):
        isim_norm.append(sum(isim[:i + 1]) / (i + 1))

    return isim_norm


# l1 = np.random.randint(50, size=20)
# l2 = np.random.randint(50, size=20)
# print(l1)
# print(l2)
# isim(l1,l2)

In [ ]:
def hits_isim(
    times=None,
    nodes=None,
    edge_weight=None,
    analyze_a=True,
    analyze_h=True,
    folder_input=['hits'],
    file_input=['networks/temporal_times.csv', 'networks/temporal_nodes.csv'],
    file_output=['output.csv'],
    input_label='',
    output_label='',
    visualized=True,
    save=True,
    top=100):
    """
    Function to calculate intersection similarity between differnt calculated HITS scores
    """
    files = []
    for file in os.listdir(HITS_PATH):
        if file.endswith('.csv') and file.startswith('A'):
            if 'score' in file:
                files.append(os.path.join(HITS_PATH, file))
    files.sort()

    isims = []
    combs = []
    comb = combinations(range(len(files)), 2)
    for i, j in comb:
        combs.append((i, j))
        # read size-2 combination of score files
        score1 = pd.read_csv(files[i]).values
        score2 = pd.read_csv(files[j]).values
        # calulate average score of nodes
        avg_1 = score1.sum(axis=1) / score1.shape[1]
        avg_2 = score2.sum(axis=1) / score2.shape[1]
        # index-sort of nodes with highest average score to lowest
        top1 = avg_1.argsort()[::-1][:top]
        top2 = avg_2.argsort()[::-1][:top]
        # calcualte  of two lists of top nodes
        isims.append(isim(top1, top2))

    fig, axes = plt.subplots(nrows=len(isims), ncols=1, figsize=(8, 6))
    for i, ax in enumerate(axes):
        x = range(1, len(isims[i]) + 1)
        y = isims[i]
        l = files[combs[i][0]].split('/')[-1].split('.')[0] + ' vs. ' + files[
            combs[i][1]].split('/')[-1].split('.')[0]
        ax.plot(x, y, label=l)
        ax.legend()
        ax.set_title('Intersection Similarity')
    fig.savefig(IMAGE_PATH + '/isim.pdf',
                dpi=300,
                bbox_inches='tight',
                transparent=True)

    print(isims)

# Centrality Robustness

$I_R=\sum_{i=1}^{N}|R^{o}_{i}-R^{n}_{i}|$

In [ ]:
size = 10
l1 = np.random.choice(size, size=size, replace=False)
rank1 = dict(zip(l1, range(len(l1))))
# or
# rank1 = {v: k for k, v in enumerate(np.random.choice(10, size=10, replace=False))}
l2 = np.random.permutation(l1)
rank2 = dict(zip(l2, range(len(l2))))
print(rank1)
print(rank2)

In [ ]:
def robustness(rank1, rank2):
    '''
    Description:
        Calculate robustness of centrality algorithm to imposed noise on network
    Parameters:
        rank1 (dict): {node_if:rank} of original network
        rank2 (dict): top ranking dictionary of noisy network
    '''
    rob = []  # robustness
    for node in sorted(rank1):
        rob.append(abs(rank1[node] - rank2[node]))
    for node, rank_change in enumerate(rob):
        print(node, ':', rank_change)
    return sum(rob)

In [ ]:
robustness(rank1, rank2)

# Supra-Matrix

In [ ]:
def supra(
    graph,
    times,
    version=0,
    parameter_omega=1,  # horizontal link weight
    parameter_epsilon=1,  # crossed link weight
    parameter_gamma=0.0001,  # horizontal link teleportation
    parameter_sigma=0.85,  # crossed link teleportation
    parameter_directed=False,
    output=False,
    output_int=False,
    save=False):
    """
    Create supra-matrix from tempral network
    v-0 Taylor paper with undirected inter-layer temporal links and default w=1
        .0 C = A eigenvector centrality
        .1 C = A^TA authorities centrality
        .2 C = AA^T hub centrality
        .3 C = (alpha * A^T * D^-1) + ((1-alpha) * N^-1 * 11^T) pageRank
    v-1: Taylor paper with directed inter-layer temporal links and default w=1 
    """

    # variables
    N = graph.number_of_nodes()
    L = graph.number_of_edges()
    T = len(times)

    # we have timestamps map via "times" which is a series index -> timestamp
    # we also need such a map for nodes i.e. index -> node_id/name
    nodes = pd.Series(sorted(list(graph.nodes())))

    S = np.zeros((N * T, N * T))  # supra-matrix
    A = np.kron(np.zeros((T, T)), np.identity(N))  # temporal coupling

    # list of ordered graph at different time
    # i.e. G = [graph(t_1),graph(t_2),...,graph(t_T)]
    G = [nx.DiGraph() for _ in range(T)]
    for u, v, w in graph.edges(data=True):
        t_index = times[times == w['t']].index[0]
        u_index = nodes[nodes == u].index[0]
        v_index = nodes[nodes == v].index[0]
        G[t_index].add_edge(u_index, v_index)

    # list of ordered centrality matrices at different time
    # i.e. C = [C(t_1),C(t_2),...,C(t_T)]
    C = []

    # add horizontal (i.e. inter-layer) edges
    if not parameter_directed:  # undirected temporal
        # (non-vectorized)
        #         for t in range(T - 1):
        #             S[N * t:N * (t + 1), N * (t + 1):N * (t + 2)] = np.identity(
        #                 (N)) * parameter_omega
        #             S[N * (t + 1):N * (t + 2), N * t:N * (t + 1)] = np.identity(
        #                 (N)) * parameter_omega
        # (vectorized)
        A = np.kron(np.eye(T, k=1) + np.eye(T, k=-1),
                    np.eye(N)) * parameter_omega
    else:  # directed
        # 1 + gamma for t'-t=1 and gamma for rest
        A = (np.kron(np.eye(T, k=1), np.eye(N)) +
             np.kron(np.ones(
                 (T, T)) * parameter_gamma, np.eye(N))) * parameter_omega

    # add crossed (i.e. intra-layer) edges
    if version == 0:  # eigenvector centrality
        # (non-vectorized)
        #         for u, v, w in graph.edges(data=True):
        #             t_index = times[times == w['t']].index[0]
        #             u_index = nodes[nodes == u].index[0]
        #             v_index = nodes[nodes == v].index[0]
        #             S[(N * t_index) + u_index,
        #               (N * t_index) + v_index] = 1 * parameter_epsilon
        # (vectorized)
        for g in G:
            C.append(nx.to_numpy_matrix(g) * parameter_epsilon)
        S = la.block_diag(*C)

    if version == 1:  # authority centrality
        for g in G:
            C.append(
                np.dot(np.transpose(nx.to_numpy_matrix(g)),
                       nx.to_numpy_matrix(g)) * parameter_epsilon)
        S = la.block_diag(*C)

    if version == 2:  # hub centrality
        for g in G:
            C.append(
                np.dot(nx.to_numpy_matrix(g),
                       np.transpose(nx.to_numpy_matrix(g))) *
                parameter_epsilon)
        S = la.block_diag(*C)

    if version == 3:  # pagerank centrality
        # D = diag[d_1,...,d_N] i.e. diagonal matrix that encodes the node out-degrees
        for g in G:
            Adj = nx.to_numpy_matrix(g)
            D_inv = np.linalg.inv(
                np.diag(np.asarray(np.sum(Adj, axis=1)).reshape(-1)))
            C.append((parameter_sigma *
                      np.dot(np.transpose(nx.to_numpy_matrix(g)), D_inv)) +
                     (((1 - parameter_sigma) / N) * np.ones((N, N))))
        S = la.block_diag(*C)

    S = S + A  # at this time S = Zeros

    if output:
        row_column_index = [node for node in nodes] * T
        print('   ', end='')
        for element in row_column_index:
            if not output_int:
                print(element, '  ', end='')  # float
            else:
                print(element, '', end='')  # int
        print()
        for i, element in enumerate(S):
            if not output_int:
                print(row_column_index[i], '', *element)  # float
            else:
                print(row_column_index[i], '', *element.astype(int))  # int

    return S

In [ ]:
# S = supra(T, times)
# S = supra(T, times, output_int=True, output=True)
# S = supra(T, times, version=2, output=True)
S = supra(T, times, version=2, parameter_directed=True, output=True)

In [ ]:
def mat_centrality(matrix,
                   times,
                   version=0,
                   parameter_negative=True,
                   parameter_normalized=True,
                   parameter_reshape=True,
                   output=False,
                   save=False):
    """
    Receive supra-matrix and calulate eigenvalues and eigenvectors
    version:
        0: general right eigenvector using numpy/scipy
        1: general left eigenvector
        2: symetric right eigenvector
    """

    T = len(times)
    N = int(S.shape[0] / T)

    eigvals = []
    eigvecs = []

    if version == 0:
        # eigvals, eigvecs = la.eig(S)
        eigvals, eigvecs = np.linalg.eig(S)
    elif version == 1:
        eigvals, eigvecs, eigvecs_right = la.eig(S, left=True)
    elif version == 2:
        eigvals, eigvecs = la.eigh(S)

    eigvals = eigvals.real

    # ind = np.argsort(eigvals)
    # eigvals = eigvals[ind]
    # eigvecs = eigvecs[:, ind]

    max_val_ind = np.argmax(eigvals)
    max_val = max(eigvals)
    max_vec = eigvecs[:, max_val_ind].reshape(-1, 1).copy()

    if parameter_negative and all(max_vec < 0):
        if output: print('eigenvector *= -1')
        max_vec *= -1

    if parameter_normalized:
        if output: print('normalizing eigenvector ...')
        max_vec /= sum(abs(max_vec))

    if parameter_reshape:
        # max_vec = max_vec.flatten().reshape((T,N)).T
        max_vec = max_vec.reshape((T, N)).T

    if output:
        # print('maximum eigenvalue is {} at index {}'.format(max_val, max_val_ind))
        # print(eigvals)
        # print(max_vec)
        matrix_print(max_vec, out_int=False)

    return max_vec

In [ ]:
mat_cen = mat_centrality(S, times, output=True)

#### Some eigenvector test

In [ ]:
# SVD
# u,s,v = np.linalg.svd(S)

In [ ]:
eigvals, eigvecs = la.eig(S)
# eigvals, eigvecs = np.linalg.eig(S)
# eigvals, eigvecs, eigvecs_right = la.eig(S, left=True)

eigvals = eigvals.real
print(eigvals)

max_index = np.argmax(eigvals)
max_value = max(eigvals)
print('max = {} at index {}'.format(max_value, max_index))
max_vec = eigvecs[:, max_index].reshape(-1, 1).copy()
if all(max_vec < 0):
    print('all of components are negative')
    max_vec *= -1
max_vec /= sum(abs(max_vec))
print(max_vec)

In [ ]:
# max_vec.flatten()
# max_vec.flatten().reshape((6,4)).T
max_vec.reshape((6, 4)).T

# Run

#### Data

In [ ]:
# dataset(output=True)

#### Temporal Network

In [ ]:
# create
# T, times = temporal_create(
#     input_label='',
#     output_label='',
#     # year_from=1936,
#     year_until=1990,  # 2018
#     filter_degree=1,
#     output_times=False,
#     output_network=True,
#     save_times=True,
#     save_nodes=True,
#     save_network=True,
#     save_edgelist=False)

# read
# T, times = temporal_read(input_label='90_1', output_network=True)
# times = times_read(input_label='90_1')
# nodes = nodes_read(input_label='90_1')

#### Static Network

In [ ]:
# create
# G = static_create(T,
#                   output_label='90_1',
#                   input_undirected=True,
#                   output_network=True,
#                   save_network=True,
#                   save_edgelist=False)

# read
# G = static_read(input_label='90_1', output_network=True)

#### DT Network -Light

In [ ]:
# create
# D = dt_create(T,
#               times,
#               output_label='90_1',
#               parameter_light=True,
#               parameter_teleport=False,
#               parameter_undirected=False,
#               parameter_loop=False,
#               output_network=True,
#               save_network=True,
#               save_edgelist=False)

# read
D = dt_read(input_label='90_1', output_network=True)

#### DT Network -Full

In [ ]:
# create
# D = dt_create(
#     T,
#     times,
#     output_label='taylor',
#     parameter_light=False,
#     version=0,
#     parameter_omega=1,
#     parameter_epsilon=1,
#     parameter_alpha=0.5,
#     parameter_distance=1,
#     parameter_teleport=True,
#     parameter_gamma=0.0001,
#     parameter_sigma=0.85,
#     parameter_undirected=True,
#     parameter_loop=True,
#     parameter_color=True,
#     parameter_delta=False,
#     parameter_delta_type='h',
#     output_delta=False,
#     output_weight=False,
#     output_network=True,
#     save_delta=False,
#     save_network=True,
#     save_edgelist=False)

# read
# D = dt_read(input_label='taylor', output_network_stat=True)

#### EW

In [ ]:
# create
# ew = ew_create(
#     D,
#     input_label='90_1',
#     output_label='90_1',
#     undirected=False,
#     omega=1,  # 1
#     epsilon=1,
#     gamma=0.0001,
#     penalize=True,  # False
#     version=3,  # 1
#     distance=1,
#     alpha=0.5,
#     save=True,
#     output=False)

# read
ew = ew_read(input_label='90_1')

#### HITS

In [ ]:
a, h = hits(
    D,
    times,
    nodes=None,
    edge_weight=ew,
    input_label='90_1',
    output_label='90_1_rand',
    max_iter=500,  # 100
    version=3,  # 1
    sigma=0.95,  # 0.85
    norm_max=True,
    norm_final=True,
    norm_iter=False,
    norm_degree=False,
    norm_damping=False,
    round_num=5,
    save=True,
    analyze=True,
    report=True)

year_until = 90, filter_node_degree = 1, epsilon = 1, omega = 0.8, penalize = False (no dynamic temporal edge weight)    
netx -> finished at 196 iterations    
book -> finished at 181 iterations    
paper -> finished at 72 iterations    
method 4 -> finished at 180 iteration and 52m    
method 5 -> finished at 4 iteration and 3m    
method 6 -> finished at 2 iteration and 2m    
***
year_until = 90, filter_node_degree = 1, epsilon = 1, omega = 0.95, penalize = False (no dynamic temporal edge weight)    
paper -> finished at 172 iterations  
***
netx -> finished at 198 iterations and 1h38m   
book -> finished at 182 iterations and 1h9m    
paper -> finished at 273 iterations and 2h40m

#### HITS Analyze

In [ ]:
df = hits_analyze(
    a,
    h,
    D,
    edge_weight=ew,
    input_label='90_1',
    output_label='',
    top=1,
    section=3,  # 1
    report_num=100,  # 1000
    round_num=5)

In [ ]:
df = hits_analyze_read()

In [ ]:
# len(df)
# df.head(6)
len(df.columns)

In [ ]:
df.iloc[:240]  # row

In [ ]:
ids = [int(idx[i].split('_')[0]) for i in range(0, len(idx), 6)]

In [ ]:
for i in range(len(ids)):
    print(i, ":", ids[i])

In [ ]:
a_nodes, h_nodes = np.array_split(
    [int(idx[i].split('_')[0]) for i in range(0, len(idx), 6)], 2)

In [ ]:
a_nodes

In [ ]:
h_nodes

#### ISIM and Robustness

In [ ]:
# hits_isim()

#### Paper Results

In [ ]:
[('b_g', 3.49), ('b_rt', 4.24), ('b_tt_1', 4.78), ('o_gy1', 5.0), ('y_rt', 5.08)]# only return the largest eigenvalue and eigenvector
# eigvals, eigvecs = la.eigh(S, lower=False, eigvals=(23, 23))
# print(eigvals[0])
# print(eigvecs[:, 0])

In [ ]:
vecc = np.array([
    0.0305, 0.0198, 0.0249, 0.0238, 0.0461, 0.0368, 0.0491, 0.0465, 0.0493,
    0.0480, 0.0592, 0.0660, 0.0460, 0.0501, 0.0520, 0.0744, 0.0360, 0.0471,
    0.0402, 0.0552, 0.0195, 0.0308, 0.0212, 0.0275
])
print(vecc.sum())
vecc = vecc.reshape((6, 4)).T
print(vecc)

sum of the all of centralities = 1 meaning the largest eigenvector is normalized with L-1